In [1]:
#Cell 1
################################################################################
#
# 🌊⚛️ LUCIDORCA ULTIMATE SOLVER - REFACTORED
#
# Cell 1: Imports & Environment Setup
#
# This cell is designed with "backwards-planning" (MDMP) based on our
# 3-Phase (Heuristic, Abstraction, Reasoning) roadmap.
#
# We are importing all dependencies for the *entire system* up front
# to ensure a clean, linear execution flow.
#
################################################################################

# --- Core Python Libraries ---
import numpy as np
import json
import time
import os
import gc
import sys
import resource
import pickle
import re
import copy
import signal
from pathlib import Path
from typing import List, Dict, Tuple, Optional, Any, Callable, Set
from dataclasses import dataclass, field
from collections import defaultdict, Counter, deque
from enum import Enum
from itertools import combinations, product

# --- Phase 1: Heuristic Triage & Object Perception ---
# scipy.ndimage.label is the S-tier replacement for our
# custom _flood_fill_label. It is C-optimized and robust.
from scipy.ndimage import label as scipy_label

# --- Phase 3: Reasoning Engine (Deep Search) ---
# Required for the true parallel search in our ReasoningSolver
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor, TimeoutError as FutureTimeoutError

# --- Environment & Logging Setup ---
# Set higher recursion depth for deep symbolic search
sys.setrecursionlimit(10000)

print("="*70)
print("🌊⚛️ LucidOrca Solver: Cell 1 Imports Loaded")
print(f"  Python Version: {sys.version.split(' ')[0]}")
print(f"  Numpy Version: {np.__version__}")
print(f"  Recursion Limit: {sys.getrecursionlimit()}")
print("="*70)
#Cell 1


🌊⚛️ LucidOrca Solver: Cell 1 Imports Loaded
  Python Version: 3.11.13
  Numpy Version: 1.26.4
  Recursion Limit: 10000


In [2]:
#Cell 2
################################################################################
#
# 🌊⚛️ LUCIDORCA ULTIMATE SOLVER - (LTM-v4 REBUILD)
#
# Cell 2: Global Configuration & Core Utilities
#
# *** LTM-v4 HOTFIX 19 ("Ultra-Deep, Hyper-Pruned" Search): ***
# 1. Rationale: We are still getting a mix of Timeout and MaxDepth.
#    We will trade "pruning width" (Cell 6) for more "search depth".
# 2. `MAX_PROGRAM_DEPTH`: Increased from 15 to 20 (to solve MaxDepth).
# 3. `BEAM_SEARCH_WIDTH`: Kept at 5 (our "Narrow" setting).
#
################################################################################

# --- 1. Global Configuration ---

@dataclass
class ChampionshipConfig:
    """
    Single source of truth for all solver parameters.
    This allows for easy tuning and validation.
    """
    
    # --- R&D / Diagnostic Mode ---
    DIAGNOSTIC_RUN: bool = True
    DIAGNOSTIC_SAMPLE_SIZE: int = 100 # Number of tasks for "micro-train"
    
    # --- HOTFIX 16 Knobs ---
    DIAGNOSTIC_MIN_RUNTIME_MINUTES: float = 30.0 # 30-minute minimum
    PUNT_TASK_BUDGET_SECONDS: float = 60.0       # 1-minute per punt task
    
    
    # --- Time Management (in seconds) ---
    total_time_budget: float = 28800.0   # 8 hours (Kaggle Hard Limit is 9)
    submission_buffer: float = 900.0     # 15 min buffer for saving files
    
    # --- LTM Training Budget (HOTFIX 11) ---
    LTM_BUDGET_PERCENT: float = 0.30     # 30% of total_time_budget
    
    # --- Time Allocation Ratios for 3-Phase Solving (Inference) ---
    abstraction_pass_time_ratio: float = 0.30 # 30% of *remaining* time
    reasoning_pass_time_ratio: float = 0.70 # 70% of *remaining* time

    # --- System & Resource ---
    parallel_workers: int = 4                # Match Kaggle's 4 CPU cores
    kaggle_memory_gb: float = 16.0
    memory_limit_ratio: float = 0.75         # Use 75% of 16GB = 12GB
    max_memory_bytes: int = int(kaggle_memory_gb * memory_limit_ratio * 1024**3)

    # --- LTM-v4 Solver Behavior Knobs ---
    
    # *** HOTFIX 19: "Ultra-Deep, Hyper-Pruned" ***
    MAX_PROGRAM_DEPTH: int = 20 # Increased from 15
    BEAM_SEARCH_WIDTH: int = 5 # Kept from HOTFIX 18
    
    # k-Nearest Neighbors for LTM cache query
    LTM_CACHE_K: int = 5


# Instantiate the global config object
CONFIG = ChampionshipConfig()

print("="*70)
print("🌊⚛️ LucidOrca Solver: Cell 2 Configuration (LTM-v4) [HOTFIX 19]")
if CONFIG.DIAGNOSTIC_RUN:
    print("  *** ⚠️  DIAGNOSTIC MODE ENABLED ⚠️ ***")
    print(f"  Sample Size: {CONFIG.DIAGNOSTIC_SAMPLE_SIZE} tasks")
    print(f"  Minimum Runtime: {CONFIG.DIAGNOSTIC_MIN_RUNTIME_MINUTES:.0f} minutes")
print(f"  Punt Task Budget: {CONFIG.PUNT_TASK_BUDGET_SECONDS:.0f} seconds")
print(f"  Total Time Budget: {CONFIG.total_time_budget / 3600:.2f} hours")
print(f"  LTM Training Budget: {CONFIG.LTM_BUDGET_PERCENT * 100:.0f}% of Total")
print(f"  Memory Limit: {CONFIG.max_memory_bytes / 1024**3:.2f} GB")
print(f"  Program Depth: {CONFIG.MAX_PROGRAM_DEPTH} (Actions) | Beam Width: {CONFIG.BEAM_SEARCH_WIDTH}")


# --- 2. Core Utility Classes ---

class TimingProfiler:
    """Track timing at every level: task, solver, function, operation"""
    def __init__(self):
        self.timings = defaultdict(list)
        self.start_times = {}
        self.call_counts = defaultdict(int)

    def start(self, category: str):
        self.start_times[category] = time.time()

    def end(self, category: str):
        if category in self.start_times:
            duration = time.time() - self.start_times[category]
            self.timings[category].append(duration)
            self.call_counts[category] += 1
            del self.start_times[category]
            return duration
        return 0.0

    def get_stats(self, category: str = None):
        if category:
            if category in self.timings:
                times = self.timings[category]
                if not times: return {'count': 0, 'total': 0, 'mean': 0, 'median': 0, 'min': 0, 'max': 0}
                return {
                    'count': len(times),
                    'total': sum(times),
                    'mean': np.mean(times),
                    'median': np.median(times),
                    'min': min(times),
                    'max': max(times),
                }
            return {}
        return {cat: self.get_stats(cat) for cat in self.timings.keys()}

    def print_summary(self, top_n: int = 20):
        print("\n" + "="*70)
        print("⏱️  DETAILED TIMING BREAKDOWN")
        print("="*70)
        
        valid_categories = {k: sum(v) for k, v in self.timings.items() if v}
        if not valid_categories:
            print("  No timing data recorded.")
            print("="*70)
            return

        sorted_categories = sorted(
            valid_categories.keys(),
            key=lambda k: valid_categories[k],
            reverse=True
        )[:top_n]
        
        for cat in sorted_categories:
            stats = self.get_stats(cat)
            print(f"  {cat:40s}: {stats['total']:7.2f}s ({stats['count']:4d} calls, "
                  f"avg: {stats['mean']:.3f}s)")
        print("="*70)

profiler = TimingProfiler()

# --- Metric Logger (HOTFIX 9) ---

class MetricLogger:
    """A simple, robust CSV logger for our meta-analysis."""
    def __init__(self, filepath: Path):
        self.filepath = filepath
        try:
            self.file_handle = open(self.filepath, 'w')
            print(f"  ✅ MetricLogger initialized. Writing to {self.filepath}")
        except Exception as e:
            print(f"  ❌ CRITICAL: MetricLogger FAILED to open file: {e}")
            self.file_handle = None

    def write_header(self, columns: List[str]):
        if not self.file_handle: return
        try:
            print(','.join(columns), file=self.file_handle, flush=True)
        except Exception as e:
            print(f"  ❌ MetricLogger Error (Header): {e}")

    def log(self, data: Dict):
        if not self.file_handle: return
        try:
            str_data = [str(data.get(k, "")) for k in data['columns_order']]
            print(','.join(str_data), file=self.file_handle, flush=True)
        except Exception as e:
            print(f"  ❌ MetricLogger Error (Log): {e}")
    
    def close(self):
        if self.file_handle:
            self.file_handle.close()
            print(f"  ✅ MetricLogger closed. Final logs saved to {self.filepath}")


# --- 3. Resource & Memory Utilities ---

def setup_memory_limits():
    """Set memory limits to % of Kaggle's kernel limit"""
    try:
        soft, hard = resource.getrlimit(resource.RLIMIT_AS)
        target_bytes = CONFIG.max_memory_bytes
        
        resource.setrlimit(resource.RLIMIT_AS, (target_bytes, hard))
        
        soft, hard = resource.getrlimit(resource.RLIMIT_AS)
        print(f"🧠 Memory limit set: {soft / (1024**3):.2f} GB")
        
        gc.enable()
        gc.set_threshold(700, 10, 10)
        print(f"♻️  Garbage collection: ENABLED (aggressive mode)")
        
    except Exception as e:
        print(f"⚠️  Could not set memory limit (not on Linux?): {e}")

def get_memory_usage() -> dict:
    """Get current memory usage statistics"""
    try:
        usage = resource.getrusage(resource.RUSAGE_SELF)
        max_rss_gb = usage.ru_maxrss / (1024 * 1024)
        return {'max_rss_gb': max_rss_gb, 'max_rss_mb': usage.ru_maxrss / 1024}
    except Exception as e:
        print(f"⚠️  Could not get memory usage: {e}")
        return {'max_rss_gb': 0, 'max_rss_mb': 0}


# --- 4. Difficulty & Time Allocation Heuristics ---

def estimate_task_difficulty(task: dict) -> float:
    """Estimate task difficulty for curriculum learning (part of Phase 1 Triage)."""
    profiler.start("estimate_task_difficulty")
    examples = task.get('train', [])
    if not examples:
        profiler.end("estimate_task_difficulty")
        return 999.0
    
    try:
        avg_grid_size = np.mean([
            np.array(ex['input']).size + np.array(ex['output']).size
            for ex in examples
        ])
        grid_complexity = avg_grid_size / 100.0

        all_colors = set()
        for ex in examples:
            all_colors.update(np.array(ex['input']).flatten().tolist())
            all_colors.update(np.array(ex['output']).flatten().tolist())
        color_complexity = len(all_colors) * 0.5

        num_examples_penalty = 10.0 / (len(examples) + 1)

        shape_changes = sum(
            1 for ex in examples
            if np.array(ex['input']).shape != np.array(ex['output']).shape
        )
        shape_complexity = shape_changes * 2.0

        size_ratios = []
        for ex in examples:
            in_size = np.array(ex['input']).size
            out_size = np.array(ex['output']).size
            if in_size > 0:
                size_ratios.append(abs(out_size / in_size - 1.0))
        size_change_complexity = np.mean(size_ratios) * 3.0 if size_ratios else 0.0

        difficulty = (
            grid_complexity +
            color_complexity +
            num_examples_penalty +
            shape_complexity +
            size_change_complexity
        )
        profiler.end("estimate_task_difficulty")
        return difficulty
    
    except Exception as e:
        profiler.end("estimate_task_difficulty")
        return 10.0

print("  Core utilities (Profiler, Memory, Difficulty, MetricLogger) defined.")
print("="*70)
#Cell 2


🌊⚛️ LucidOrca Solver: Cell 2 Configuration (LTM-v4) [HOTFIX 19]
  *** ⚠️  DIAGNOSTIC MODE ENABLED ⚠️ ***
  Sample Size: 100 tasks
  Minimum Runtime: 30 minutes
  Punt Task Budget: 60 seconds
  Total Time Budget: 8.00 hours
  LTM Training Budget: 30% of Total
  Memory Limit: 12.00 GB
  Program Depth: 20 (Actions) | Beam Width: 5
  Core utilities (Profiler, Memory, Difficulty, MetricLogger) defined.


In [3]:
#Cell 3
################################################################################
#
# 🌊⚛️ LUCIDORCA ULTIMATE SOLVER - (LTM-v4 REBUILD)
#
# Cell 3: The Perception Engine & Atomic Primitives
#
# *** LTM-v4 REFACTOR: ***
# 1. This cell's primary responsibility is now PERCEPTION.
# 2. `HyperFeatureObjectClustering` (from old Cell 5/7) is promoted
#    to be the core `PerceptionEngine`.
# 3. `HyperObject` (the "noun" of our AGI) is now defined here.
# 4. DEPRECATED: `find_abstraction_rule` is REMOVED. It is a failed model.
# 5. The old primitives (e.g., `rot90`) are re-classified as
#    `ATOMIC_PRIMITIVES`. They are "building blocks" for the
#    new meta-primitives (Cell 4), not solvers in themselves.
#
################################################################################

print("="*70)
print("🌊⚛️ LucidOrca Solver: Cell 3 (LTM-v4) Perception Engine")

# --- 1. The Core "Noun" of our AGI ---

@dataclass
class HyperObject:
    """
    A "noun" with advanced features, used by all LTM-v4+ systems.
    This dataclass is the primary output of the PerceptionEngine.
    """
    obj_id: int
    color: int
    size: int
    positions: np.ndarray  # (N, 2) array of (r, c) coordinates
    bbox: Tuple[int, int, int, int] # (min_r, min_c, max_r, max_c)
    center: Tuple[float, float]     # (mean_r, mean_c)
    
    # Advanced features (from LTM-v2)
    symmetry_score: float = 0.0
    density: float = 0.0
    hierarchy_level: int = 0
    topology: str = "simple" # 'simple', 'hollow', etc.

print("  Defined: HyperObject (The AGI's 'Noun')")


# --- 2. The Core "Vision System" of our AGI ---

class PerceptionEngine:
    """
    This is the core LTM-v4 "Vision System."
    It replaces and merges all previous perception modules.
    Its job is to take a raw grid and transform it into a
    structured list of HyperObjects.
    
    This is the foundation for Blind Spot #1 (Salience) and #2 (Multi-Modal).
    """
    def __init__(self):
        print("  PerceptionEngine (LTM-v4 'Vision') initialized.")
        # This class is stateless for now
        pass

    def analyze(self, grid: np.ndarray) -> List[HyperObject]:
        """
        Analyzes a grid and returns a list of all found HyperObjects.
        """
        profiler.start("PerceptionEngine.analyze")
        
        # Use scipy.ndimage.label for C-optimized object finding
        basic_objects = self._extract_basic_objects(grid)
        if not basic_objects:
            profiler.end("PerceptionEngine.analyze")
            return []

        # Compute advanced features for each object
        hyper_objects = [self._compute_hyper_features(obj, i, grid) 
                         for i, obj in enumerate(basic_objects)]
        
        # Compute relational features (e.g., "is_inside")
        self._compute_hierarchy(hyper_objects)
        
        profiler.end("PerceptionEngine.analyze")
        return hyper_objects

    def _extract_basic_objects(self, grid: np.ndarray) -> List[Dict]:
        """
        Uses scipy.ndimage.label to find all connected components ("blobs").
        """
        objects = []
        if grid.size == 0:
            return objects
        
        unique_colors = np.unique(grid)
        for color in unique_colors:
            if color == 0: continue # Ignore background
            
            color_mask = (grid == color)
            labeled_array, num_features = scipy_label(color_mask)

            for obj_id in range(1, num_features + 1):
                obj_mask = (labeled_array == obj_id)
                positions = np.argwhere(obj_mask)
                if positions.size == 0: continue
                
                min_row, min_col = positions.min(axis=0)
                max_row, max_col = positions.max(axis=0)

                objects.append({
                    'color': int(color),
                    'size': len(positions),
                    'positions': positions,
                    'mask': obj_mask,
                    'bbox': (min_row, min_col, max_row, max_col),
                    'center': (positions[:, 0].mean(), positions[:, 1].mean()),
                })
        return objects

    def _compute_hyper_features(self, obj: Dict, obj_id: int, grid: np.ndarray) -> HyperObject:
        """
        Upgrades a "basic object" dict to a "HyperObject" dataclass
        by computing advanced features (density, symmetry, etc.).
        """
        min_r, min_c, max_r, max_c = obj['bbox']
        bbox_area = (max_r - min_r + 1) * (max_c - min_c + 1)
        
        # Get the snippet of the grid corresponding to the object
        obj_region = grid[min_r:max_r+1, min_c:max_c+1]
        obj_mask_local = obj['mask'][min_r:max_r+1, min_c:max_c+1]
        obj_grid_snippet = obj_region * obj_mask_local

        # Calculate local symmetry
        symmetry_h = np.mean(obj_grid_snippet == np.fliplr(obj_grid_snippet)) if obj_grid_snippet.size > 0 else 0.0
        symmetry_v = np.mean(obj_grid_snippet == np.flipud(obj_grid_snippet)) if obj_grid_snippet.size > 0 else 0.0
        
        density = obj['size'] / max(bbox_area, 1)
        
        return HyperObject(
            obj_id=obj_id,
            color=obj['color'],
            positions=obj['positions'],
            size=obj['size'],
            bbox=obj['bbox'],
            center=obj['center'],
            symmetry_score=(symmetry_h + symmetry_v) / 2.0,
            density=density,
            topology="hollow" if bbox_area > 0 and density < 0.5 and density > 0.1 else "simple"
        )

    def _compute_hierarchy(self, objects: List[HyperObject]):
        """
        Calculates relational features, like "which object is inside which".
        Modifies objects in-place.
        """
        for i, obj_i in enumerate(objects):
            level = 0
            for j, obj_j in enumerate(objects):
                if i == j: continue
                
                # Is obj_i's center inside obj_j's bbox?
                min_r, min_c, max_r, max_c = obj_j.bbox
                cy, cx = obj_i.center
                if min_r < cy < max_r and min_c < cx < max_c:
                    # More advanced check: is it *truly* contained?
                    # For now, a simple bbox check is a good heuristic.
                    level += 1
            obj_i.hierarchy_level = level

print("  Defined: PerceptionEngine (Core LTM-v4 'Vision' System)")


# --- 3. Atomic Primitive Dictionaries ("The Building Blocks") ---
# These are the "legacy" grid-level ops. They are no longer
# solvers, but are the instruction set for our LTM-v5 "Math Brain"
# and the components of our LTM-v4 "Meta-Primitives" (Cell 4).

# Geometric primitives
ATOMIC_PRIMITIVES_GEOMETRIC = {
    'identity': lambda g: g,
    'rot90': lambda g: np.rot90(g, 1),
    'rot180': lambda g: np.rot90(g, 2),
    'rot270': lambda g: np.rot90(g, 3),
    'flip_h': lambda g: np.fliplr(g), # Flip horizontal (left-right)
    'flip_v': lambda g: np.flipud(g), # Flip vertical (up-down)
    'transpose': lambda g: g.T,
    'anti_transpose': lambda g: np.rot90(g.T, 2),
}

# Scaling primitives
ATOMIC_PRIMITIVES_SCALING = {
    'tile_2x2': lambda g: np.tile(g, (2, 2)),
    'tile_2x1': lambda g: np.tile(g, (2, 1)),
    'tile_1x2': lambda g: np.tile(g, (1, 2)),
    'tile_3x3': lambda g: np.tile(g, (3, 3)),
}

# Color primitives
ATOMIC_PRIMITIVES_COLOR = {
    'invert_colors_mod10': lambda g: (9 - g) % 10,
    'increment_colors_mod10': lambda g: (g + 1) % 10,
    'decrement_colors_mod10': lambda g: (g - 1) % 10,
    'mask_nonzero': lambda g: (g > 0).astype(int),
    'extract_color_1': lambda g: (g == 1).astype(int),
    'extract_color_2': lambda g: (g == 2).astype(int),
    'extract_color_3': lambda g: (g == 3).astype(int),
    'extract_color_4': lambda g: (g == 4).astype(int),
    'extract_color_5': lambda g: (g == 5).astype(int),
    'extract_color_6': lambda g: (g == 6).astype(int),
    'extract_color_7': lambda g: (g == 7).astype(int),
    'extract_color_8': lambda g: (g == 8).astype(int),
}

# Spatial & Morphological primitives
ATOMIC_PRIMITIVES_SPATIAL = {
    'center_crop_1px': lambda g: g[1:-1, 1:-1] if g.shape[0] > 2 and g.shape[1] > 2 else g,
    'border_pad_1px_zero': lambda g: np.pad(g, 1, mode='constant', constant_values=0),
    'extract_edges': lambda g: (np.abs(g - np.roll(g, 1, axis=0)) + 
                                np.abs(g - np.roll(g, 1, axis=1))) > 0,
}

# Aggregation/Value primitives (output grid is a single value)
ATOMIC_PRIMITIVES_AGGREGATION = {
    'count_unique_colors': lambda g: np.array([[len(np.unique(g[g > 0]))]]),
    'count_pixels': lambda g: np.array([[np.sum(g > 0)]]),
    'get_max_color': lambda g: np.array([[np.max(g)]]),
}

# All "atomic" primitives combined
ALL_ATOMIC_PRIMITIVES = {
    **ATOMIC_PRIMITIVES_GEOMETRIC,
    **ATOMIC_PRIMITIVES_SCALING,
    **ATOMIC_PRIMITIVES_COLOR,
    **ATOMIC_PRIMITIVES_SPATIAL,
    **ATOMIC_PRIMITIVES_AGGREGATION,
}

# Pre-compute the list of (name, function) tuples for fast iteration
atomic_primitives_to_test = list(ALL_ATOMIC_PRIMITIVES.items())

print(f"  Defined: {len(ALL_ATOMIC_PRIMITIVES)} 'Atomic Primitives' (Building Blocks)")
print("="*70)
#Cell 3


🌊⚛️ LucidOrca Solver: Cell 3 (LTM-v4) Perception Engine
  Defined: HyperObject (The AGI's 'Noun')
  Defined: PerceptionEngine (Core LTM-v4 'Vision' System)
  Defined: 30 'Atomic Primitives' (Building Blocks)


In [4]:
#Cell 4
################################################################################
#
# 🌊⚛️ LUCIDORCA ULTIMATE SOLVER - (LTM-v4 REBUILD)
#
# Cell 4: The Meta-Primitive Instruction Set (The "ALU")
#
# *** LTM-v4 HOTFIX 15 (HPN Prereq): ***
# 1. This is the "full-power" ALU, including `draw_path` (from HOTFIX 12),
#    which our new HPN-guided synthesizer will now learn to use
#    intelligently instead of by "flailing".
#
################################################################################

print("="*70)
print("🌊⚛️ LucidOrca Solver: Cell 4 (LTM-v4 HOTFIX 15) Meta-Primitive Set")

# --- 1. The LTM-v4 "Instruction Set Architecture" (ISA) ---

class MetaPrimitives:
    """
    Container for our new "Object-Aware" (LTM-v4) primitive instruction set.
    """
    
    def __init__(self, perception_engine: PerceptionEngine):
        """
        The Meta-Primitive set *requires* a PerceptionEngine (from Cell 3)
        to "see" the objects it will act upon.
        """
        self.perception_engine = perception_engine
        print("  MetaPrimitives (LTM-v4 'ALU') initialized. [HOTFIX 15]")

    # --- "Finder" / "Selector" Primitives (The "Nouns") ---
    
    def find_objects(self, ctx: Dict, grid: np.ndarray, **params) -> Tuple[Dict, np.ndarray]:
        """
        Stores List[HyperObject] in `ctx['last_result']`.
        Returns (new_ctx, original_grid).
        """
        profiler.start("Primitive.find_objects")
        all_objects = self.perception_engine.analyze(grid)
        filtered_objects = all_objects
        
        if 'color' in params:
            filtered_objects = [o for o in filtered_objects if o.color == params['color']]
        if 'size' in params:
            filtered_objects = [o for o in filtered_objects if o.size == params['size']]
        if 'min_size' in params:
            filtered_objects = [o for o in filtered_objects if o.size >= params['min_size']]
        if 'max_size' in params:
            filtered_objects = [o for o in filtered_objects if o.size <= params['max_size']]
        if 'topology' in params:
            filtered_objects = [o for o in filtered_objects if o.topology == params['topology']]
        if 'hierarchy' in params:
            filtered_objects = [o for o in filtered_objects if o.hierarchy_level == params['hierarchy']]
            
        ctx['last_result'] = filtered_objects
        profiler.end("Primitive.find_objects")
        return ctx, grid # Return original grid

    def get_largest(self, ctx: Dict, grid: np.ndarray, **params) -> Tuple[Dict, np.ndarray]:
        """Takes a list of objects from `ctx` and returns only the largest."""
        target_key = params.get('target', 'last_result')
        objects = ctx.get(target_key, [])
        if not objects or not isinstance(objects, list):
            ctx['last_result'] = []
            return ctx, grid
            
        max_size = max(o.size for o in objects)
        ctx['last_result'] = [o for o in objects if o.size == max_size]
        return ctx, grid

    def get_smallest(self, ctx: Dict, grid: np.ndarray, **params) -> Tuple[Dict, np.ndarray]:
        """Takes a list of objects from `ctx` and returns only the smallest."""
        target_key = params.get('target', 'last_result')
        objects = ctx.get(target_key, [])
        if not objects or not isinstance(objects, list):
            ctx['last_result'] = []
            return ctx, grid
            
        min_size = min(o.size for o in objects)
        ctx['last_result'] = [o for o in objects if o.size == min_size]
        return ctx, grid


    # --- "Transformer" / "Action" Primitives (The "Verbs") ---
    
    def move(self, ctx: Dict, grid: np.ndarray, **params) -> Tuple[Dict, np.ndarray]:
        """
        Moves a list of objects (from `ctx`) by a given `delta` (dr, dc).
        Returns (new_ctx, new_grid).
        """
        profiler.start("Primitive.move")
        target_key = params.get('target', 'last_result')
        delta = params.get('delta', (0, 0))
        dr, dc = delta
        objects_to_move = ctx.get(target_key, [])
        
        if not objects_to_move or not isinstance(objects_to_move, list):
            profiler.end("Primitive.move")
            return ctx, grid
            
        moved_pixels = set()
        for obj in objects_to_move:
            if isinstance(obj, HyperObject):
                for r, c in obj.positions:
                    moved_pixels.add((r, c))

        new_grid = np.zeros_like(grid)
        rows, cols = grid.shape
        
        for r in range(rows):
            for c in range(cols):
                if (r, c) not in moved_pixels:
                    new_grid[r, c] = grid[r, c]
                    
        for obj in objects_to_move:
            if isinstance(obj, HyperObject):
                for r, c in obj.positions:
                    new_r, new_c = r + dr, c + dc
                    if 0 <= new_r < rows and 0 <= new_c < cols:
                        new_grid[new_r, new_c] = obj.color
        
        ctx['last_result'] = new_grid
        profiler.end("Primitive.move")
        return ctx, new_grid

    def delete(self, ctx: Dict, grid: np.ndarray, **params) -> Tuple[Dict, np.ndarray]:
        """
        Deletes a list of objects (from `ctx`) from the grid.
        Returns (new_ctx, new_grid).
        """
        profiler.start("Primitive.delete")
        target_key = params.get('target', 'last_result')
        objects_to_delete = ctx.get(target_key, [])
        
        if not objects_to_delete or not isinstance(objects_to_delete, list):
            profiler.end("Primitive.delete")
            return ctx, grid
            
        new_grid = grid.copy()
        for obj in objects_to_delete:
             if isinstance(obj, HyperObject):
                for r, c in obj.positions:
                    new_grid[r, c] = 0
        
        ctx['last_result'] = new_grid
        profiler.end("Primitive.delete")
        return ctx, new_grid

    def recolor(self, ctx: Dict, grid: np.ndarray, **params) -> Tuple[Dict, np.ndarray]:
        """
        Changes the color of a list of objects (from `ctx`) to a `new_color`.
        Returns (new_ctx, new_grid).
        """
        profiler.start("Primitive.recolor")
        target_key = params.get('target', 'last_result')
        new_color = params.get('color', 0)
        objects_to_recolor = ctx.get(target_key, [])
        
        if not objects_to_recolor or not isinstance(objects_to_recolor, list):
            profiler.end("Primitive.recolor")
            return ctx, grid
            
        new_grid = grid.copy()
        for obj in objects_to_recolor:
            if isinstance(obj, HyperObject):
                for r, c in obj.positions:
                    new_grid[r, c] = new_color
        
        ctx['last_result'] = new_grid
        profiler.end("Primitive.recolor")
        return ctx, new_grid
    
    def copy_objects(self, ctx: Dict, grid: np.ndarray, **params) -> Tuple[Dict, np.ndarray]:
        """
        Copies a list of objects (from `ctx`) by a given `delta` (dr, dc).
        The original objects are *kept* in place.
        Returns (new_ctx, new_grid).
        """
        profiler.start("Primitive.copy_objects")
        
        target_key = params.get('target', 'last_result')
        delta = params.get('delta', (0, 0))
        dr, dc = delta
        
        objects_to_copy = ctx.get(target_key, [])
        if not objects_to_copy or not isinstance(objects_to_copy, list):
            profiler.end("Primitive.copy_objects")
            return ctx, grid
            
        new_grid = grid.copy()
        rows, cols = grid.shape
                    
        for obj in objects_to_copy:
             if isinstance(obj, HyperObject):
                for r, c in obj.positions:
                    new_r, new_c = r + dr, c + dc
                    if 0 <= new_r < rows and 0 <= new_c < cols:
                        new_grid[new_r, new_c] = obj.color
        
        ctx['last_result'] = new_grid
        profiler.end("Primitive.copy_objects")
        return ctx, new_grid

    
    # --- Drawing Primitive (from HOTFIX 12) ---
    
    def _bresenham_line(self, grid: np.ndarray, p1: Tuple[int, int], 
                        p2: Tuple[int, int], color: int):
        """
        Internal helper. Draws a line on `grid` in-place.
        p1, p2 are (row, col) tuples.
        """
        x0, y0 = p1[1], p1[0] # (col, row)
        x1, y1 = p2[1], p2[0] # (col, row)
        rows, cols = grid.shape

        dx = abs(x1 - x0)
        dy = -abs(y1 - y0)
        sx = 1 if x0 < x1 else -1
        sy = 1 if y0 < y1 else -1
        err = dx + dy
        
        while True:
            if 0 <= y0 < rows and 0 <= x0 < cols:
                grid[y0, x0] = color
            if x0 == x1 and y0 == y1:
                break
            e2 = 2 * err
            if e2 >= dy:
                err += dy
                x0 += sx
            if e2 <= dx:
                err += dx
                y0 += sy

    def draw_path(self, ctx: Dict, grid: np.ndarray, **params) -> Tuple[Dict, np.ndarray]:
        """
        Takes a list of objects, sorts them by y-coordinate, and draws
        a path connecting their centers.
        """
        profiler.start("Primitive.draw_path")
        
        target_key = params.get('target', 'all_objects')
        color = params.get('color', 1)
        if color == 0: color = 1
            
        objects_to_connect = ctx.get(target_key, [])
        if not objects_to_connect or not isinstance(objects_to_connect, list) or len(objects_to_connect) < 2:
            profiler.end("Primitive.draw_path")
            return ctx, grid
            
        new_grid = grid.copy()
        
        sorted_objects = sorted(objects_to_connect, key=lambda o: (o.center[0], o.center[1]))
        
        for i in range(len(sorted_objects) - 1):
            p1 = (int(round(sorted_objects[i].center[0])), int(round(sorted_objects[i].center[1])))
            p2 = (int(round(sorted_objects[i+1].center[0])), int(round(sorted_objects[i+1].center[1])))
            
            self._bresenham_line(new_grid, p1, p2, color)
            
        ctx['last_result'] = new_grid
        profiler.end("Primitive.draw_path")
        return ctx, new_grid

    
print("  Defined: MetaPrimitives (The LTM-v4 'ALU' / Instruction Set) [HOTFIX 15]")
print("="*70)
#Cell 4


🌊⚛️ LucidOrca Solver: Cell 4 (LTM-v4 HOTFIX 15) Meta-Primitive Set
  Defined: MetaPrimitives (The LTM-v4 'ALU' / Instruction Set) [HOTFIX 15]


In [5]:
#Cell 5
################################################################################
#
# 🌊⚛️ LUCIDORCA ULTIMATE SOLVER - (LTM-v4 REBUILD)
#
# Cell 5: Cognitive Fingerprinting & Triage Engine
#
# *** LTM-v4 HOTFIX 11 (Punt Strategy Prereq): ***
# 1. `TaskProfile` dataclass: Added `difficulty_score: float` field.
# 2. `TaskAnalyzer.analyze`: Now saves the raw difficulty score to the
#    profile. This is required by Cell 9 to sort tasks and find the
#    9 "easiest" targets for our 2x budget "punt".
#
################################################################################

print("="*70)
print("🌊⚛️ LucidOrca Solver: Cell 5 (LTM-v4) Fingerprinting & Triage [HOTFIX 11]")

# --- 1. The "Task Passport" Dataclass ---

@dataclass
class TaskProfile:
    """
    A "passport" for each task, generated by the TaskAnalyzer in Phase 1.
    It holds all metadata needed for solving, including the LTM-v4 "probe".
    """
    task_id: str
    difficulty_tier: str  # 'easy', 'medium', 'hard'
    basin: str            # 'rotation', 'color_mapping', 'scaling', 'unknown'
    
    # --- The LTM-v4 "Probe" ---
    delta_fingerprint: Optional[np.ndarray] = None
    
    # --- HOTFIX 11: Store raw score for sorting ---
    difficulty_score: float = 0.0

print("  Defined: TaskProfile (The AGI's 'Task Passport')")


# --- 2. The Core Grid-to-Vector Encoder ---

class VisionModelEncoder:
    """
    An "unrolled" vision model. It extracts statistical and object-based
    features from a single grid and encodes them into a normalized
    1D numpy vector (the "fingerprint").
    """
    
    def __init__(self, perception_engine: PerceptionEngine):
        """
        The encoder *requires* the PerceptionEngine from Cell 3 to "see"
        objects and calculate object-based statistics.
        """
        self.perception_engine = perception_engine
        self.zero_vector = np.zeros(self.get_vector_dim())
        print(f"  VisionModelEncoder (LTM-v4) initialized. Fingerprint dim: {self.get_vector_dim()}")

    def get_vector_dim(self) -> int:
        """Returns the total dimension of our feature vector."""
        # shape(2) + color_hist(10) + obj_stats(4) + symmetry(3) + 
        # patterns(4) + layout(4) + complexity(1) = 28 dimensions
        return 28

    def encode_grid_to_vector(self, grid: np.ndarray) -> np.ndarray:
        """
        Encodes a single grid into a normalized 1D numpy vector ("fingerprint").
        """
        profiler.start("VisionModelEncoder.encode_grid_to_vector")
        
        if grid.size == 0:
            profiler.end("VisionModelEncoder.encode_grid_to_vector")
            return self.zero_vector 
            
        edge_density_val = self._compute_edge_density(grid)
        
        objects = self.perception_engine.analyze(grid)
        obj_stats = self._compute_object_stats(objects, grid.size)
        
        shape_norm = np.array([grid.shape[0], grid.shape[1]]) / 30.0
        color_hist = np.bincount(grid.flatten(), minlength=10) / max(1.0, grid.size)
        edge_density = np.array([edge_density_val])
        symmetry_vec = self._detect_symmetry(grid, as_vector=True)
        patterns_vec = self._match_patterns(grid, as_vector=True)
        layout_vec = self._analyze_layout(grid, as_vector=True)
        complexity = np.array([self._compute_complexity(grid, edge_density_val, obj_stats[0])])

        vector = np.concatenate([
            shape_norm,      # 2 dims
            color_hist,      # 10 dims
            obj_stats,       # 4 dims
            symmetry_vec,    # 3 dims
            patterns_vec,    # 4 dims
            layout_vec,      # 4 dims
            complexity       # 1 dim
        ])
        
        profiler.end("VisionModelEncoder.encode_grid_to_vector")
        return np.nan_to_num(vector, nan=0.0, posinf=0.0, neginf=0.0)

    # --- ENCODER HELPER METHODS ---

    def _compute_object_stats(self, objects: List[HyperObject], grid_size: int) -> np.ndarray:
        """Calculates normalized statistics about the object population."""
        if not objects:
            return np.zeros(4) # count, avg_size, avg_density, num_colors
            
        obj_count = len(objects)
        avg_size = np.mean([o.size for o in objects])
        avg_density = np.mean([o.density for o in objects])
        num_colors = len(set(o.color for o in objects))
        
        norm_obj_count = min(obj_count / 100.0, 1.0)
        norm_avg_size = min(avg_size / (grid_size + 1e-6), 1.0)
        norm_avg_density = avg_density
        norm_num_colors = min(num_colors / 10.0, 1.0)
        
        return np.array([norm_obj_count, norm_avg_size, norm_avg_density, norm_num_colors])

    @staticmethod
    def _compute_edge_density(grid: np.ndarray) -> float:
        if grid.size == 0: return 0.0
        h_edges = np.sum(np.abs(np.diff(grid, axis=0)))
        v_edges = np.sum(np.abs(np.diff(grid, axis=1)))
        return (h_edges + v_edges) / max(grid.size * 9, 1) 

    @staticmethod
    def _detect_symmetry(grid: np.ndarray, as_vector: bool = False) -> Any:
        if grid.size == 0: 
            return np.zeros(3) if as_vector else []
            
        symmetries = []
        is_h, is_v, is_d = 0.0, 0.0, 0.0
        try:
            if np.array_equal(grid, np.flipud(grid)): 
                symmetries.append('horizontal'); is_h = 1.0
            if np.array_equal(grid, np.fliplr(grid)): 
                symmetries.append('vertical'); is_v = 1.0
            if grid.shape[0] == grid.shape[1]:
                if np.array_equal(grid, grid.T): 
                    symmetries.append('diag_main'); is_d = 1.0
        except Exception: pass
        
        return np.array([is_h, is_v, is_d]) if as_vector else symmetries

    def _match_patterns(self, grid: np.ndarray, as_vector: bool = False) -> Any:
        if grid.size == 0:
            return np.zeros(4) if as_vector else []
            
        patterns = []
        is_stripe, is_grid_p, is_binary, is_sparse = 0.0, 0.0, 0.0, 0.0
        try:
            if self._is_stripe_pattern(grid): 
                patterns.append('stripe'); is_stripe = 1.0
            if self._is_grid_pattern(grid): 
                patterns.append('grid'); is_grid_p = 1.0
            if len(np.unique(grid)) <= 2: 
                patterns.append('binary_color'); is_binary = 1.0
            if np.sum(grid == 0) > grid.size * 0.8: 
                patterns.append('sparse'); is_sparse = 1.0
        except Exception: pass
        
        return np.array([is_stripe, is_grid_p, is_binary, is_sparse]) if as_vector else patterns

    @staticmethod
    def _is_stripe_pattern(grid: np.ndarray) -> bool:
        if grid.shape[0] < 2 or grid.shape[1] < 2: return False
        h_stripe = all(len(np.unique(row)) == 1 for row in grid)
        v_stripe = all(len(np.unique(col)) == 1 for col in grid.T)
        return h_stripe or v_stripe

    @staticmethod
    def _is_grid_pattern(grid: np.ndarray) -> bool:
        if grid.size < 4: return False
        nonzero = np.argwhere(grid != 0)
        rows, cols = nonzero[:, 0], nonzero[:, 1]
        row_diffs, col_diffs = np.diff(np.sort(np.unique(rows))), np.diff(np.sort(np.unique(cols)))
        row_regular = (len(row_diffs) > 0) and (len(np.unique(row_diffs)) == 1)
        col_regular = (len(col_diffs) > 0) and (len(np.unique(col_diffs)) == 1)
        return row_regular and col_regular

    @staticmethod
    def _analyze_layout(grid: np.ndarray, as_vector: bool = False) -> Any:
        is_empty, is_centered, is_scattered, is_distributed = 0.0, 0.0, 0.0, 1.0
        layout_str = 'distributed'

        if grid.size == 0: 
            is_empty, is_distributed = 1.0, 0.0
            layout_str = 'empty'
        else:
            nonzero = np.argwhere(grid != 0)
            if len(nonzero) == 0:
                is_empty, is_distributed = 1.0, 0.0
                layout_str = 'empty'
            else:
                centroid = nonzero.mean(axis=0)
                center_h, center_w = (grid.shape[0] - 1) / 2, (grid.shape[1] - 1) / 2
                dist = np.linalg.norm(centroid - np.array([center_h, center_w]))
                spread = nonzero.std(axis=0).mean()
                
                max_dim = max(grid.shape[0], grid.shape[1], 1.0)
                
                if dist < max_dim * 0.2 and spread < max_dim * 0.3:
                    is_centered, is_distributed = 1.0, 0.0
                    layout_str = 'centered'
                elif spread > max_dim * 0.4:
                    is_scattered, is_distributed = 1.0, 0.0
                    layout_str = 'scattered'

        return np.array([is_empty, is_centered, is_scattered, is_distributed]) if as_vector else layout_str

    @staticmethod
    def _compute_complexity(grid: np.ndarray, edge_density: float, obj_count: float) -> float:
        """Computes a single 'complexity' score [0, 1]."""
        if grid.size == 0: return 0.0
        
        flat = grid.flatten()
        _, counts = np.unique(flat, return_counts=True)
        probs = counts / len(flat)
        entropy = -np.sum(probs * np.log2(probs + 1e-10))
        max_entropy = np.log2(len(counts)) if len(counts) > 1 else 1
        entropy_normalized = entropy / max(1, max_entropy)
        
        return (edge_density * 0.3) + (obj_count * 0.3) + (entropy_normalized * 0.4)

print("  Defined: VisionModelEncoder (Grid-to-Vector Engine)")


# --- 3. The Task-to-Delta-Vector Encoder ---

class TaskFingerprinter:
    """
    Computes the "delta-fingerprint" for an entire task.
    This vector represents the *abstract transformation* of the task
    (i.e., v_output - v_input).
    """
    def __init__(self, vision_encoder: VisionModelEncoder):
        self.encoder = vision_encoder
        self.zero_vector = self.encoder.zero_vector
        print("  TaskFingerprinter (Task-to-Delta-Vector Engine) initialized.")

    def fingerprint(self, task: Dict) -> np.ndarray:
        """
        Computes the average delta-vector (v_output - v_input)
        across all training examples for a task.
        """
        profiler.start("TaskFingerprinter.fingerprint")
        examples = task.get('train', [])
        if not examples:
            profiler.end("TaskFingerprinter.fingerprint")
            return self.zero_vector

        delta_vectors = []
        for ex in examples:
            try:
                inp_grid = np.array(ex['input'])
                out_grid = np.array(ex['output'])
                
                v_in = self.encoder.encode_grid_to_vector(inp_grid)
                v_out = self.encoder.encode_grid_to_vector(out_grid)
                
                v_delta = v_out - v_in
                delta_vectors.append(v_delta)
            
            except Exception:
                continue 
        
        if not delta_vectors:
            profiler.end("TaskFingerprinter.fingerprint")
            return self.zero_vector
            
        mean_delta = np.mean(delta_vectors, axis=0)
        profiler.end("TaskFingerprinter.fingerprint")
        return mean_delta

print("  Defined: TaskFingerprinter (The 'Probe' Generator)")


# --- 4. The Phase 1 Triage Engine ---

class TaskAnalyzer:
    """
    This is the complete Phase 1 "Heuristic Triage" engine.
    Its job is to run a *fast, non-solving* analysis on every task
    to generate a TaskProfile.
    """
    def __init__(self, perception_engine: PerceptionEngine):
        self.encoder = VisionModelEncoder(perception_engine)
        self.fingerprinter = TaskFingerprinter(self.encoder)
        print("  TaskAnalyzer (Phase 1 Triage Engine) initialized.")

    def analyze(self, task: Dict, task_id: str) -> TaskProfile:
        """
        Runs all heuristic analyses on a single task and returns its profile.
        
        *** HOTFIX 11: Now saves the `difficulty_score`. ***
        """
        profiler.start(f"TaskAnalyzer.analyze.{task_id}")
        
        # 1. Get Difficulty Score (from Cell 2)
        difficulty_score = estimate_task_difficulty(task)
        if difficulty_score < 7.0:
            tier = 'easy'
        elif difficulty_score < 18.0:
            tier = 'medium'
        else:
            tier = 'hard'

        # 2. Detect Attractor Basin (Task Type)
        basin = self._detect_basin(task.get('train', []))
        
        # 3. --- LTM-v4 STEP: Generate Delta-Fingerprint ---
        try:
            delta_fingerprint = self.fingerprinter.fingerprint(task)
        except Exception:
            delta_fingerprint = None 

        # 4. Create the "Task Passport"
        profile = TaskProfile(
            task_id=task_id,
            difficulty_tier=tier,
            basin=basin,
            delta_fingerprint=delta_fingerprint,
            difficulty_score=difficulty_score # <-- *** NEW (HOTFIX 11) ***
        )
        
        profiler.end(f"TaskAnalyzer.analyze.{task_id}")
        return profile

    def _detect_basin(self, train_examples: List[Dict]) -> str:
        """Heuristically determines the 'type' of task (a fast, weak heuristic)."""
        if not train_examples:
            return 'unknown'
        
        features = {'has_shape_change': False, 'has_color_change': False, 
                    'has_object_count_change': False}
        
        for example in train_examples:
            try:
                inp, out = np.array(example['input']), np.array(example['output'])
                if inp.shape != out.shape: 
                    features['has_shape_change'] = True
                if not np.array_equal(inp, out):
                    features['has_color_change'] = True
            except Exception:
                continue
        
        if features['has_shape_change']: return 'scaling'
        if features['has_color_change']: return 'color_mapping'
        return 'unknown'

print("  Defined: TaskAnalyzer (The Triage Engine)")
print("="*70)
#Cell 5


🌊⚛️ LucidOrca Solver: Cell 5 (LTM-v4) Fingerprinting & Triage [HOTFIX 11]
  Defined: TaskProfile (The AGI's 'Task Passport')
  Defined: VisionModelEncoder (Grid-to-Vector Engine)
  Defined: TaskFingerprinter (The 'Probe' Generator)
  Defined: TaskAnalyzer (The Triage Engine)


In [6]:
#Cell 6
################################################################################
#
# 🌊⚛️ LUCIDORCA ULTIMATE SOLVER - (LTM-v4 REBUILD)
#
# Cell 6: The Cognitive Engine (CPU & Mind)
#
# *** LTM-v4 HOTFIX 19 ("Hyper-Pruning"): ***
#
# 1. Rationale: Our "Slow Brain" is still timing out. We will
#    prune its action space even more aggressively to "pay" for
#    the new `depth=20` search.
# 2. `TOP_K_ACTIONS`: Decreased from 20 to 15.
# 3. This forces the HPN-guided search to be even more "focused"
#    and less "creative," maximizing speed.
#
################################################################################

print("="*70)
print("🌊⚛️ LucidOrca Solver: Cell 6 (LTM-v4 HOTFIX 19) The Cognitive Engine")

# --- 1. The "CPU" - Executes Abstract Programs (ASTs) ---

class SymbolicProgramInterpreter:
    """
    The LTM-v4 "CPU" (Interpreter).
    """
    
    def __init__(self, perception_engine: PerceptionEngine):
        # The "ALU" (Instruction Set) from Cell 4
        self.primitives = MetaPrimitives(perception_engine)
        
        # This "dispatch table" maps 'op' strings from the AST
        # to the actual Python functions in our MetaPrimitives class.
        self.dispatch_table: Dict[str, Callable] = {
            # --- Finder Ops ---
            'find': self.primitives.find_objects,
            'get_largest': self.primitives.get_largest,
            'get_smallest': self.primitives.get_smallest,
            
            # --- Transformer Ops ---
            'move': self.primitives.move,
            'delete': self.primitives.delete,
            'recolor': self.primitives.recolor,
            'copy': self.primitives.copy_objects,
            
            # --- HOTFIX 12 Primitive ---
            'draw_path': self.primitives.draw_path,
            
            # --- Procedural Op (Solves Blind Spot #5) ---
            'map': self._op_map 
        }
        print("  SymbolicProgramInterpreter (LTM-v4 'CPU') initialized. [HOTFIX 19]")

    def run_instruction(self, ctx: Dict, grid: np.ndarray, 
                        instruction: Dict) -> Tuple[Dict, np.ndarray]:
        op_name = instruction.get('op')
        params = instruction.get('params', {})
        
        if op_name not in self.dispatch_table:
            raise ValueError(f"Unknown operation: {op_name}")
            
        op_function = self.dispatch_table[op_name]
        
        try:
            new_ctx, new_grid = op_function(copy.deepcopy(ctx), grid, **params)
            return new_ctx, new_grid
        except Exception as e:
            raise e
            
    def run(self, program_ast: List[Dict], initial_grid: np.ndarray) -> np.ndarray:
        profiler.start("Interpreter.run")
        ctx = {'last_result': None}
        current_grid = initial_grid.copy()
        try:
            for instruction in program_ast:
                (ctx, current_grid) = self.run_instruction(ctx, current_grid, instruction)
            profiler.end("Interpreter.run")
            return current_grid
        except Exception as e:
            profiler.end("Interpreter.run")
            return initial_grid

    def _op_map(self, ctx: Dict, grid: np.ndarray, **params) -> Tuple[Dict, np.ndarray]:
        target_key = params.get('target', 'last_result')
        sub_program = params.get('program', []) 
        
        if not sub_program: return ctx, grid 
        items_to_map = ctx.get(target_key, [])
        if not items_to_map or not isinstance(items_to_map, list): return ctx, grid 
        
        map_results = []
        current_grid = grid 
        for item in list(items_to_map):
            local_ctx = {'this': item, 'last_result': item}
            try:
                for instruction in sub_program:
                    (local_ctx, current_grid) = self.run_instruction(
                        local_ctx, current_grid, instruction
                    )
                map_results.append(local_ctx.get('last_result'))
            except Exception:
                continue
        ctx['last_result'] = map_results
        return ctx, current_grid

print("  Defined: SymbolicProgramInterpreter (The 'CPU') [HOTFIX 19]")


# --- 2. The "Mind" - Generates Abstract Programs (ASTs) ---

@dataclass
class BeamEntry:
    """A single state in our Beam Search."""
    program_ast: List[Dict]
    last_ctx: Dict
    current_grid: np.ndarray
    cost: float
    
    def __hash__(self):
        return hash(self.current_grid.tobytes())
    def __eq__(self, other):
        return self.current_grid.tobytes() == other.current_grid.tobytes()

class SymbolicProgramSynthesizer:
    """
    The LTM-v4 "Mind" (Solver).
    
    *** HOTFIX 19: Implements Hyper-Pruning. ***
    """
    
    # --- *** NEW (HOTFIX 19) *** ---
    # We are pruning *even more* to "pay" for depth=20.
    TOP_K_ACTIONS: int = 15 # Was 20
    
    def __init__(self, interpreter: SymbolicProgramInterpreter, 
                 cfg: ChampionshipConfig, 
                 fingerprinter: TaskFingerprinter,
                 hpn_playbook_vectors: Optional[np.ndarray],
                 hpn_playbook_programs: List,
                 hpn_grammar: Dict):
        
        self.interpreter = interpreter
        self.config = cfg
        
        # --- *** (HOTFIX 17) *** ---
        self.fingerprinter = fingerprinter
        self.hpn_playbook_vectors = hpn_playbook_vectors # The "quiz" fingerprints
        self.hpn_playbook_programs = hpn_playbook_programs # The "seed" architectures
        self.hpn_grammar = hpn_grammar # The "transition matrix"
        
        if self.hpn_playbook_vectors is not None and self.hpn_grammar:
            print("  SymbolicProgramSynthesizer (LTM-v4 'Slow Brain') initialized. [HOTFIX 19 - HPN LOADED]")
        else:
            print("  SymbolicProgramSynthesizer (LTM-v4 'Slow Brain') initialized. [HOTFIX 19 - HPN *NOT* LOADED]")

    def _heuristic(self, grid: np.ndarray, target_grid: np.ndarray) -> float:
        """The "cost" function for our search."""
        if grid.shape != target_grid.shape:
            return 1000.0 + abs(grid.size - target_grid.size)
        return np.sum(grid != target_grid)

    def _precompute_finders(self, grid: np.ndarray, base_ctx: Dict = None) -> Dict:
        """
        (From HOTFIX 7)
        Runs a static, PRUNED set of "Finder" ops to populate the context.
        """
        profiler.start("Synthesizer._precompute_finders")
        
        if base_ctx:
            ctx = copy.deepcopy(base_ctx)
        else:
            ctx = {'last_result': None}

        all_objects = []
        try:
            (find_ctx, _) = self.interpreter.run_instruction(
                ctx, grid, {'op': 'find', 'params': {}}
            )
            all_objects = find_ctx.get('last_result', [])
            ctx['all_objects'] = all_objects
        except Exception:
            ctx['all_objects'] = []
        
        find_ctx_for_derivation = {'last_result': all_objects}
        try:
            (largest_ctx, _) = self.interpreter.run_instruction(
                find_ctx_for_derivation, grid, 
                {'op': 'get_largest', 'params': {'target': 'last_result'}}
            )
            ctx['largest'] = largest_ctx.get('last_result', [])
        except Exception:
            ctx['largest'] = []

        try:
            (smallest_ctx, _) = self.interpreter.run_instruction(
                find_ctx_for_derivation, grid, 
                {'op': 'get_smallest', 'params': {'target': 'last_result'}}
            )
            ctx['smallest'] = smallest_ctx.get('last_result', [])
        except Exception:
            ctx['smallest'] = []

        profiler.end("Synthesizer._precompute_finders")
        return ctx

    def _get_possible_ops(self, ctx: Dict) -> List[Dict]:
        """
        (Unchanged from HOTFIX 17)
        Generates the "full-power" action space.
        """
        profiler.start("Synthesizer._get_possible_ops")
        possible_ops = []
        
        target_keys = [
            k for k, v in ctx.items() 
            if isinstance(v, list) and k != 'last_result' and v
        ]
        
        if not target_keys:
            profiler.end("Synthesizer._get_possible_ops")
            return []
            
        deltas = [(0, 1), (0, -1), (1, 0), (-1, 0)]
        
        for target_key in target_keys:
            # --- 1. Simple "Global" Transformer Ops (Always On) ---
            for color in range(10):
                possible_ops.append(
                    {'op': 'recolor', 'params': {'target': target_key, 'color': color}}
                )
            
            for d in deltas:
                possible_ops.append({'op': 'move', 'params': {'target': target_key, 'delta': d}})
                possible_ops.append({'op': 'copy', 'params': {'target': target_key, 'delta': d}})
                
            possible_ops.append({'op': 'delete', 'params': {'target': target_key}})
            
            # --- 2. Drawing Primitives (HOTFIX 12) ---
            for color in range(1, 10): # Iterate 1-9 (non-background)
                possible_ops.append(
                    {'op': 'draw_path', 'params': {'target': target_key, 'color': color}}
                )
        
            # --- 3. Procedural Ops (ForEach loop) ---
            map_sub_programs = []
            map_target = 'this'
            
            for color in range(10):
                map_sub_programs.append(
                    [{'op': 'recolor', 'params': {'target': map_target, 'color': color}}]
                )
            map_sub_programs.append([{'op': 'delete', 'params': {'target': map_target}}])
            
            for d in deltas:
                map_sub_programs.append([{'op': 'move', 'params': {'target': map_target, 'delta': d}}])
                map_sub_programs.append([{'op': 'copy', 'params': {'target': map_target, 'delta': d}}])
            
            for sub_prog in map_sub_programs:
                possible_ops.append({
                    'op': 'map',
                    'params': {
                        'target': target_key,
                        'program': sub_prog
                    }
                })
        
        profiler.end("Synthesizer._get_possible_ops")
        return possible_ops

    def _get_playbook_seeds(self, task: Dict) -> List[BeamEntry]:
        """
        (Unchanged from HOTFIX 17)
        Phase A ("Diagnose & Seed")
        """
        profiler.start("Synthesizer.get_playbook_seeds")
        seed_entries = []
        
        # 1. Diagnose: Fingerprint the new task
        try:
            task_fingerprint = self.fingerprinter.fingerprint(task)
        except Exception:
            profiler.end("Synthesizer.get_playbook_seeds")
            return [] 

        # 2. k-NN Lookup: Find closest "quiz" tasks
        if (self.hpn_playbook_vectors is None or 
            len(self.hpn_playbook_programs) == 0):
            profiler.end("Synthesizer.get_playbook_seeds")
            return []

        try:
            distances = np.linalg.norm(self.hpn_playbook_vectors - task_fingerprint, axis=1)
            k = 3 
            nearest_indices = np.argsort(distances)[:k]
        except Exception:
            profiler.end("Synthesizer.get_playbook_seeds")
            return []

        # 3. Load Architectures: Build new BeamEntry objects from these seeds
        initial_grid = np.array(task['train'][0]['input'])
        target_grid = np.array(task['train'][0]['output'])
        
        for i in nearest_indices:
            seed_program = self.hpn_playbook_programs[i]
            
            try:
                seed_ctx = {'last_result': None}
                seed_grid = initial_grid.copy()
                
                for instruction in seed_program:
                    (seed_ctx, seed_grid) = self.interpreter.run_instruction(
                        seed_ctx, seed_grid, instruction
                    )
                
                final_seed_ctx = self._precompute_finders(seed_grid, base_ctx=seed_ctx)
                seed_cost = self._heuristic(seed_grid, target_grid)

                seed_entries.append(BeamEntry(
                    program_ast=seed_program,
                    last_ctx=final_seed_ctx,
                    current_grid=seed_grid,
                    cost=seed_cost
                ))
            except Exception:
                continue

        profiler.end("Synthesizer.get_playbook_seeds")
        return seed_entries

    def solve(self, task: Dict, timeout: float) -> Tuple[Optional[List[Dict]], str]:
        """
        *** HOTFIX 19: Implements "Hyper-Pruning" (Top-K=15). ***
        """
        profiler.start("SymbolicProgramSynthesizer.solve")
        task_start_time = time.time()
        
        examples = task.get('train', [])
        if not examples:
            profiler.end("SymbolicProgramSynthesizer.solve")
            return None, "Synthesizer.NoTrainData"

        try:
            initial_grid = np.array(examples[0]['input'])
            target_grid = np.array(examples[0]['output'])
        except Exception:
            profiler.end("SymbolicProgramSynthesizer.solve")
            return None, "Synthesizer.BadData"
            
        initial_cost = self._heuristic(initial_grid, target_grid)
        if initial_cost == 0:
            return [], "Synthesizer.Identity"
        
        
        # --- Phase A ("Playbook Seeding") ---
        beam = self._get_playbook_seeds(task)
        
        initial_ctx = self._precompute_finders(initial_grid)
        beam.append(BeamEntry(
            program_ast=[],
            last_ctx=initial_ctx,
            current_grid=initial_grid,
            cost=initial_cost
        ))
        
        visited_states: Dict[bytes, float] = {
            entry.current_grid.tobytes(): entry.cost for entry in beam
        }

        # --- Phase B ("Grammar-Guided & Pruned Search") ---
        
        for depth in range(self.config.MAX_PROGRAM_DEPTH):
            if (time.time() - task_start_time) > timeout:
                profiler.end("SymbolicProgramSynthesizer.solve")
                return None, "Synthesizer.Timeout.Coarse"
                
            new_beam = []
            
            for entry_idx, entry in enumerate(beam):
                
                if len(entry.program_ast) >= self.config.MAX_PROGRAM_DEPTH:
                    continue
                
                if entry_idx % (max(1, self.config.BEAM_SEARCH_WIDTH // 10)) == 0:
                     if (time.time() - task_start_time) > timeout:
                        profiler.end("SymbolicProgramSynthesizer.solve")
                        return None, "Synthesizer.Timeout.Granular"
                
                # --- HPN Guidance (Level 2: "Grammar") ---
                
                possible_ops = self._get_possible_ops(entry.last_ctx)
                
                if not entry.program_ast:
                    last_op_name = "START"
                else:
                    last_op_name = entry.program_ast[-1].get('op', 'unknown')
                
                hpn_probabilities = self.hpn_grammar.get(last_op_name, {})
                
                prioritized_ops = sorted(
                    possible_ops,
                    key=lambda op: hpn_probabilities.get(op.get('op'), 0.0),
                    reverse=True
                )
                
                # --- *** NEW (HOTFIX 19): HYPER-PRUNING *** ---
                ops_to_evaluate = prioritized_ops[:self.TOP_K_ACTIONS]
                
                
                for op_ast in ops_to_evaluate: # <-- Iterate over the *pruned* list
                    try:
                        new_program = entry.program_ast + [op_ast]
                        
                        (temp_ctx, new_grid) = self.interpreter.run_instruction(
                            entry.last_ctx, 
                            entry.current_grid, 
                            op_ast
                        )
                        
                        new_ctx = self._precompute_finders(new_grid, base_ctx=temp_ctx)

                        new_grid_hash = new_grid.tobytes()
                        new_cost = self._heuristic(new_grid, target_grid)
                        
                        if visited_states.get(new_grid_hash, float('inf')) <= new_cost:
                            continue
                        visited_states[new_grid_hash] = new_cost

                        if new_cost == 0.0:
                            if self._validate_program(new_program, examples):
                                profiler.end("SymbolicProgramSynthesizer.solve")
                                return new_program, f"Synthesizer.Success.d{len(new_program)}"
                        
                        new_beam.append(BeamEntry(new_program, new_ctx, new_grid, new_cost))
                    
                    except Exception:
                        continue 
            
            if not new_beam:
                break 
            
            combined_beam = new_beam + beam
            combined_beam.sort(key=lambda e: e.cost)
            
            seen_grids = set()
            unique_beam = []
            for entry in combined_beam:
                grid_hash = entry.current_grid.tobytes()
                if grid_hash not in seen_grids:
                    unique_beam.append(entry)
                    seen_grids.add(grid_hash)
            
            beam = unique_beam[:self.config.BEAM_SEARCH_WIDTH]

        profiler.end("SymbolicProgramSynthesizer.solve")
        return None, "Synthesizer.Fail.MaxDepth"

    def _validate_program(self, program_ast: List[Dict], examples: List[Dict]) -> bool:
        """
        Validates a candidate program against ALL training examples.
        """
        for ex in examples:
            try:
                inp_grid = np.array(ex['input'])
                out_grid = np.array(ex['output'])
                
                predicted_grid = self.interpreter.run(program_ast, inp_grid)
                
                if not np.array_equal(predicted_grid, out_grid):
                    return False
            except Exception:
                return False 
                
        return True

print("  Defined: SymbolicProgramSynthesizer (The 'Mind') [HOTFIX 19]")
print("="*70)
#Cell 6


🌊⚛️ LucidOrca Solver: Cell 6 (LTM-v4 HOTFIX 19) The Cognitive Engine
  Defined: SymbolicProgramInterpreter (The 'CPU') [HOTFIX 19]
  Defined: SymbolicProgramSynthesizer (The 'Mind') [HOTFIX 19]


In [7]:
#Cell 7
################################################################################
#
# 🌊⚛️ LUCIDORCA ULTIMATE SOLVER - (LTM-v4 REBUILD)
#
# Cell 7: The Master Toolbox (Orchestrator & Assembly)
#
# *** LTM-v4 HOTFIX 17 (The *True* "Two-Brain" Assembly): ***
#
# 1. This is the synthesis of HOTFIX 15 and 16.
# 2. `LucidOrcaUltimateSolver.__init__`: Now accepts *all* HPN
#    components (playbook, grammar, etc.) from Cell 9.
# 3. `HeuristicPlaybookSolver` ("Fast Brain"): Is instantiated and
#    receives the "Playbook" HPN (L1).
# 4. `SymbolicProgramSynthesizer` ("Slow Brain"): Is now *also*
#    instantiated with *all* HPN components. It is now a
#    "schooled" searcher, not a "dumb" one.
#
################################################################################

print("="*70)
print("🌊⚛️ LucidOrca Solver: Cell 7 (LTM-v4) Master Toolbox & Assembly [HOTFIX 17]")

# --- 1. Consensus Utility (for Blind Spot #10) ---

class SolverAgreementEnsemble:
    """
    Measures solver agreement. This is a utility for our final step
    to "judge" the *results* (grids) from the Abstraction (LTM-v4 Query)
    and Reasoning (LTM-v4 Search) passes.
    """
    def __init__(self):
        self.agreement_history = []
        print("  SolverAgreementEnsemble (Consensus Utility) initialized.")

    def measure_agreement(self, grids: List[np.ndarray]) -> Tuple[float, np.ndarray]:
        if not grids:
            return 0.0, np.array([[0]])

        grid_hashes = []
        valid_grids = []
        
        for grid in grids:
            if grid is not None and grid.size > 0:
                grid_hashes.append(grid.tobytes())
                valid_grids.append(grid)
            else:
                grid_hashes.append("NONE")
        
        if not valid_grids:
             return 0.0, np.array([[0]])

        counts = Counter(grid_hashes)
        most_common_bytes, most_common_count = counts.most_common(1)[0]
        
        agreement_ratio = most_common_count / len(grid_hashes)

        if most_common_bytes != "NONE":
            collapsed_solution = next(g for g in valid_grids if g.tobytes() == most_common_bytes)
        else:
            if len(valid_grids) > 0:
                collapsed_solution = valid_grids[0]
            else:
                collapsed_solution = np.array([[0]])

        self.agreement_history.append(agreement_ratio)
        return agreement_ratio, collapsed_solution

print("  Defined: SolverAgreementEnsemble (Consensus Utility)")


# --- "Brain 1" (The "Fast" Architect - HOTFIX 16) ---

class HeuristicPlaybookSolver:
    """
    This is our "fast" brain ("Brain 1"). It is the "schooled"
    architect. It does *not* do a beam search. It only
    checks its "playbook" of known-good architectures.
    """
    def __init__(self, fingerprinter: TaskFingerprinter, 
                 interpreter: SymbolicProgramInterpreter,
                 hpn_playbook_vectors: Optional[np.ndarray],
                 hpn_playbook_programs: List):
        
        self.fingerprinter = fingerprinter
        self.interpreter = interpreter
        self.hpn_playbook_vectors = hpn_playbook_vectors
        self.hpn_playbook_programs = hpn_playbook_programs
        
        if self.hpn_playbook_vectors is not None:
            print("  HeuristicPlaybookSolver (LTM-v4 'Fast Brain') initialized. [HPN LOADED]")
        else:
            print("  HeuristicPlaybookSolver (LTM-v4 'Fast Brain') initialized. [HPN *NOT* LOADED]")

    def _validate_program(self, program_ast: List[Dict], examples: List[Dict]) -> bool:
        """
        Validates a candidate program against ALL training examples.
        """
        for ex in examples:
            try:
                inp_grid = np.array(ex['input'])
                out_grid = np.array(ex['output'])
                predicted_grid = self.interpreter.run(program_ast, inp_grid)
                if not np.array_equal(predicted_grid, out_grid):
                    return False
            except Exception:
                return False 
        return True

    def solve(self, task: Dict, timeout: float) -> Tuple[Optional[List[Dict]], str]:
        """
        This is the "fast-pass" cognitive loop:
        1. Diagnose (fingerprint)
        2. k-NN Lookup (check HPN Playbook)
        3. Validate (run the 5 best-guess architectures)
        """
        profiler.start("HeuristicPlaybookSolver.solve")
        
        examples = task.get('train', [])
        if not examples:
            profiler.end("HeuristicPlaybookSolver.solve")
            return None, "Playbook.NoTrainData"
            
        # 1. Diagnose: Fingerprint the new task
        try:
            task_fingerprint = self.fingerprinter.fingerprint(task)
        except Exception:
            profiler.end("HeuristicPlaybookSolver.solve")
            return None, "Playbook.Fail.Fingerprint"

        # 2. k-NN Lookup: Find 5 closest "quiz" architectures
        if (self.hpn_playbook_vectors is None or 
            len(self.hpn_playbook_programs) == 0):
            profiler.end("HeuristicPlaybookSolver.solve")
            return None, "Playbook.Fail.NoHPN"

        try:
            distances = np.linalg.norm(self.hpn_playbook_vectors - task_fingerprint, axis=1)
            k = 5 # Check the top 5 "plays"
            nearest_indices = np.argsort(distances)[:k]
        except Exception:
            profiler.end("HeuristicPlaybookSolver.solve")
            return None, "Playbook.Fail.KNN"

        # 3. Validate: Run the "Playbook"
        for i in nearest_indices:
            seed_program = self.hpn_playbook_programs[i]
            
            # Check if this "seed" program *as-is* solves the task
            if self._validate_program(seed_program, examples):
                profiler.end("HeuristicPlaybookSolver.solve")
                return seed_program, "Playbook.Success"
        
        # If no "play" worked...
        profiler.end("HeuristicPlaybookSolver.solve")
        return None, "Playbook.Fail.NoMatch"

print("  Defined: HeuristicPlaybookSolver (The 'Fast Brain') [HOTFIX 17]")


# --- 2. The LTM-v4 Master "Toolbox" Class (Assembly) ---

class LucidOrcaUltimateSolver:
    """
    The main "Factory Manager" or "Toolbox" class for LTM-v4.
    
    *** HOTFIX 17: Now assembles the *true* "Two-Brain" model. ***
    """
    def __init__(self, cfg: ChampionshipConfig, 
                 hpn_playbook_vectors: Optional[np.ndarray],
                 hpn_playbook_programs: List,
                 hpn_grammar: Dict): # <-- *** CORRECTED (HOTFIX 17) ***
        
        print("\n" + "="*70)
        print("🌊⚛️ Initializing LucidOrcaUltimateSolver (LTM-v4 Master Toolbox)...")
        
        self.config = cfg
        
        # --- 1. Instantiate Core "Vision" (from Cell 3) ---
        self.perception_engine = PerceptionEngine()
        print(f"  ✅ 1. PerceptionEngine (Vision)... LOADED")

        # --- 2. Instantiate "Triage & Fingerprinting" (from Cell 5) ---
        self.analyzer = TaskAnalyzer(self.perception_engine)
        self.fingerprinter = self.analyzer.fingerprinter
        print(f"  ✅ 2. TaskAnalyzer (Triage & Fingerprinting)... LOADED")

        # --- 3. Instantiate The "CPU" (from Cell 6) ---
        self.interpreter = SymbolicProgramInterpreter(self.perception_engine)
        print(f"  ✅ 3. SymbolicProgramInterpreter ('CPU')... LOADED")

        # --- 4. Instantiate The "Fast Brain" (Brain 1) ---
        self.heuristic_solver = HeuristicPlaybookSolver(
            self.fingerprinter,
            self.interpreter,
            hpn_playbook_vectors,
            hpn_playbook_programs
        )
        print(f"  ✅ 4. HeuristicPlaybookSolver ('Fast Brain')... LOADED")

        # --- 5. Instantiate The "Schooled Slow Brain" (Brain 2) ---
        # *** CORRECTED (HOTFIX 17): We now pass *all* HPN components to
        # the "slow" brain, making it a "schooled" searcher.
        self.synthesizer = SymbolicProgramSynthesizer(
            self.interpreter, 
            self.config, 
            self.fingerprinter,
            hpn_playbook_vectors,
            hpn_playbook_programs,
            hpn_grammar
        )
        print(f"  ✅ 5. SymbolicProgramSynthesizer ('Schooled Slow Brain')... LOADED")
        
        # --- 6. Instantiate Utilities ---
        self.utilities = {
            'agreement_ensemble': SolverAgreementEnsemble()
        }
        print("  ✅ 6. Consensus & Utility Modules... LOADED")

        print("\n✅ LucidOrca LTM-v4 Solver (Toolbox) is ready!"); print("="*70)

print("="*70)
#Cell 7


🌊⚛️ LucidOrca Solver: Cell 7 (LTM-v4) Master Toolbox & Assembly [HOTFIX 17]
  Defined: SolverAgreementEnsemble (Consensus Utility)
  Defined: HeuristicPlaybookSolver (The 'Fast Brain') [HOTFIX 17]


In [8]:
#Cell 8
################################################################################
#
# 🌊⚛️ LUCIDORCA ULTIMATE SOLVER - (LTM-v4 REBUILD)
#
# Cell 8: Load All Task Data
#
# This cell defines the data loading functions and loads all necessary
# JSON files from the Kaggle environment.
#
# 1. `test_tasks`: The set of tasks we must solve for submission.
# 2. `training_tasks` & `evaluation_tasks`: The 1120 known tasks
#    that our "Game Genie" (Cell 9) will use to "train" (build its LTM).
# 3. `training_solutions` & `evaluation_solutions`: The ground-truth
#    solutions required by Cell 9 to *validate* the rules it finds.
#
################################################################################

print("="*70)
print("🌊⚛️ LucidOrca Solver: Cell 8 (LTM-v4) Load Task Data")

# --- 1. Define Data Paths ---\n# This assumes the standard Kaggle competition dataset path
DATA_DIR = Path("/kaggle/input/arc-prize-2025")

# The "unknown" tasks for our final submission
TEST_CHALLENGES_PATH = DATA_DIR / "arc-agi_test_challenges.json"

# The "known" tasks for our Game Genie (LTM)
TRAINING_CHALLENGES_PATH = DATA_DIR / "arc-agi_training_challenges.json"
EVALUATION_CHALLENGES_PATH = DATA_DIR / "arc-agi_evaluation_challenges.json"

# The "ground truth" for our Game Genie (LTM)
TRAINING_SOLUTIONS_PATH = DATA_DIR / "arc-agi_training_solutions.json"
EVALUATION_SOLUTIONS_PATH = DATA_DIR / "arc-agi_evaluation_solutions.json"


# --- 2. Data Loading Function ---\n
def load_json_tasks(file_path: Path) -> Dict[str, Dict]:
    """
    Loads a JSON task file from the given path.
    Returns a dictionary of {task_id: task_data}.
    """
    if not file_path.exists():
        print(f"  ⚠️  WARNING: Task file not found at: {file_path}")
        print(f"     Please ensure the 'ARC Prize 2025' dataset is added to this notebook.")
        return {}
    
    try:
        profiler.start(f"load_json.{file_path.name}")
        with open(file_path, 'r') as f:
            tasks = json.load(f)
        profiler.end(f"load_json.{file_path.name}")
        
        print(f"  ✅ Loaded {len(tasks)} tasks from {file_path.name}")
        return tasks
        
    except Exception as e:
        print(f"  ❌ CRITICAL ERROR: Could not load or parse {file_path.name}: {e}")
        return {}

# --- 3. Load the Test Set (Primary Goal) ---\nprint("\nLoading test set for submission...")
test_tasks = load_json_tasks(TEST_CHALLENGES_PATH)

if test_tasks:
    print(f"\nSample task IDs from test set:")
    for i, task_id in enumerate(list(test_tasks.keys())[:3]):
        try:
            task = test_tasks[task_id]
            print(f"   {i+1}. {task_id}: {len(task.get('test', []))} test cases")
        except Exception as e:
            print(f"   {i+1}. {task_id}: Error parsing task info - {e}")
else:
    print("  No test tasks loaded. Submission will not be possible.")

# --- 4. Load Training/Evaluation Data (for "Game Genie" in Cell 9) ---
print("\nLoading supplemental data for 'Game Genie' LTM pre-computation...")
training_tasks = load_json_tasks(TRAINING_CHALLENGES_PATH)
training_solutions = load_json_tasks(TRAINING_SOLUTIONS_PATH)
evaluation_tasks = load_json_tasks(EVALUATION_CHALLENGES_PATH)
evaluation_solutions = load_json_tasks(EVALUATION_SOLUTIONS_PATH)

print("="*70)
#Cell 8


🌊⚛️ LucidOrca Solver: Cell 8 (LTM-v4) Load Task Data
  ✅ Loaded 240 tasks from arc-agi_test_challenges.json

Sample task IDs from test set:
   1. 00576224: 1 test cases
   2. 007bbfb7: 1 test cases
   3. 009d5c81: 1 test cases

Loading supplemental data for 'Game Genie' LTM pre-computation...
  ✅ Loaded 1000 tasks from arc-agi_training_challenges.json
  ✅ Loaded 1000 tasks from arc-agi_training_solutions.json
  ✅ Loaded 120 tasks from arc-agi_evaluation_challenges.json
  ✅ Loaded 120 tasks from arc-agi_evaluation_solutions.json


In [9]:
#Cell 9
################################################################################
#
# 🌊⚛️ LUCIDORCA ULTIMATE SOLVER - (LTM-v4 REBUILD)
#
# Cell 9: RSC Controller & "Game Genie" LTM-v4 Trainer
#
# *** LTM-v4 HOTFIX 19 (The *True* "Brute-Force Time" Test): ***
#
# 1. Rationale: The HOTFIX 18 run proved our "Fast Brain" (Playbook)
#    was "cheating" on the 10 "Punt" tasks, solving them in 0.01s
#    and preventing the "Schooled Slow Brain" from ever running.
# 2. `run_pre_computation`: Now passes `is_punt_task=True` to the
#    solver loop for the 10 "Punt" tasks.
# 3. `_find_ground_truth_program`:
#    - Now accepts an `is_punt_task` flag.
#    - **IF `is_punt_task` IS TRUE:** It **SKIPS** the "Fast Brain"
#      and *forces* the "Schooled Slow Brain" (`SymbolicProgramSynthesizer`)
#      to run with the full 60-second budget.
#    - **ELSE:** It uses the normal "Fast Brain -> Slow Brain"
#      fallback logic for the "Standard" tasks.
#
################################################################################

print("="*70)
print("🌊⚛️ LucidOrca Solver: Cell 9 (LTM-v4) RSC Controller & LTM Trainer [HOTFIX 19]")

# --- 1. Define the RSC Controller (LTM-v4) ---

class RSC_Controller:
    """
    Implements the Recursive Symbolic Coherence (RSC) Framework.
    *** HOTFIX 19: Manages the "True Two-Brain" HPN architecture. ***
    """
    
    # --- LTM Training Budget (HOTFIX 16) ---
    LTM_BUDGET_PERCENT: float = CONFIG.LTM_BUDGET_PERCENT
    LTM_NUM_TASKS: float = 1120.0 # 1000 train + 120 eval
    
    # --- Log Headers (HOTFIX 9) ---
    LTM_LOG_COLUMNS = [
        "phase", "task_id", "task_tier", "status", 
        "time_taken_s", "program_cached"
    ]
    INFERENCE_LOG_COLUMNS = [
        "phase", "task_id", "task_tier", "basin", 
        "ltm_status", "reasoning_status", "final_program", "time_taken_s"
    ]
    
    def __init__(self, cfg: ChampionshipConfig):
        print("\nInitializing RSC_Controller (LTM-v4)...")
        self.config = cfg
        
        self.log_path = Path("/kaggle/working/lucid_metrics.csv")
        self.metric_logger = MetricLogger(self.log_path)
        
        # --- *** (HOTFIX 17.1) *** ---
        # 1. Build a *temporary* "Toolbox" just to get the fingerprinter
        print("  Bootstrapping Cognitive Architecture (Phase 1)...")
        bootcamp_toolbox = LucidOrcaUltimateSolver(cfg, None, [], {}) 
        
        # 2. Run the bootcamp to build the HPN "Playbook"
        print("  Bootstrapping Cognitive Architecture (Phase 2)...")
        profiler.start("GameGenie.Bootcamp")
        (
            self.hpn_playbook_vectors,
            self.hpn_playbook_programs,
            self.hpn_grammar
        ) = self._run_architectural_bootcamp(bootcamp_toolbox.fingerprinter)
        profiler.end("GameGenie.Bootcamp")
        print(f"  ✅ Cognitive Bootcamp complete. Built HPN Grammar ({len(self.hpn_grammar)} rules) "
              f"and HPN Playbook ({len(self.hpn_playbook_programs)} seeds).")
        
        # 3. Build the *real* "Two-Brain" Toolbox
        self.solver_toolbox = LucidOrcaUltimateSolver(
            cfg,
            self.hpn_playbook_vectors,
            self.hpn_playbook_programs,
            self.hpn_grammar
        )
        
        # --- LTM-v4: "Game Genie" Long-Term Memory ---
        self._ltm_vectors_list: List[np.ndarray] = []
        self.ltm_programs: List[List[Dict]] = [] 
        self.ltm_vectors: Optional[np.ndarray] = None 
        self.task_profiles: Dict[str, TaskProfile] = {}
        self.time_allocations = {
            'abstraction_per_task': {},
            'reasoning_per_task': {},
        }
        print("  ✅ RSC_Controller is online (LTM-v4 Architecture).")

    def _run_architectural_bootcamp(self, fingerprinter: TaskFingerprinter
                                   ) -> Tuple[Optional[np.ndarray], List, Dict]:
        """
        *** (Unchanged from HOTFIX 17.1): This is the "School". ***
        
        Builds both HPNs (Level 1 Playbook, Level 2 Grammar).
        """
        
        # 1. The "Curriculum" (Core Architectures)
        CURRICULUM = [
            [{'op': 'find'}, {'op': 'recolor'}], [{'op': 'find'}, {'op': 'move'}],
            [{'op': 'find'}, {'op': 'copy'}], [{'op': 'find'}, {'op': 'delete'}],
            [{'op': 'get_largest'}, {'op': 'recolor'}], [{'op': 'get_smallest'}, {'op': 'move'}],
            [{'op': 'get_largest'}, {'op': 'copy'}], [{'op': 'get_smallest'}, {'op': 'delete'}],
            [{'op': 'find'}, {'op': 'draw_path'}], [{'op': 'get_largest'}, {'op': 'draw_path'}],
            [{'op': 'find'}, {'op': 'map', 'program': [{'op': 'recolor'}]}],
            [{'op': 'find'}, {'op': 'map', 'program': [{'op': 'move'}]}],
            [{'op': 'find'}, {'op': 'map', 'program': [{'op': 'delete'}]}],
            [{'op': 'copy'}, {'op': 'find'}, {'op': 'recolor'}],
            [{'op': 'move'}, {'op': 'find'}, {'op': 'delete'}],
            [{'op': 'draw_path'}, {'op': 'find'}, {'op': 'recolor'}],
            [{'op': 'map'}, {'op': 'find'}, {'op': 'move'}], [{'op': 'find'}]
        ]

        # 2. The "Pop Quiz" (Atomic Tasks for Playbook)
        QUIZ_TASKS = {
            'recolor_all': {'task': {'train': [{'input': [[1]], 'output': [[2]]}]}, 
                            'arch': [{'op': 'find'}, {'op': 'recolor', 'params': {'color': 2}}]},
            'move_all': {'task': {'train': [{'input': [[1,0]], 'output': [[0,1]]}]},
                         'arch': [{'op': 'find'}, {'op': 'move', 'params': {'delta': (0,1)}}]},
            'copy_all': {'task': {'train': [{'input': [[1,0]], 'output': [[1,1]]}]},
                         'arch': [{'op': 'find'}, {'op': 'copy', 'params': {'delta': (0,1)}}]},
            'delete_all': {'task': {'train': [{'input': [[1]], 'output': [[0]]}]},
                           'arch': [{'op': 'find'}, {'op': 'delete'}]},
            'draw_path': {'task': {'train': [{'input': [[1,0,1]], 'output': [[1,5,1]]}]},
                          'arch': [{'op': 'find'}, {'op': 'draw_path', 'params': {'color': 5}}]},
            'map_delete': {'task': {'train': [{'input': [[1,2,1]], 'output': [[2]]}]}, # Deletes color 1
                           'arch': [{'op': 'find', 'params': {'color': 1}}, 
                                    {'op': 'map', 'program': [{'op': 'delete'}]}]},
            'move_largest': {'task': {'train': [{'input': [[1,0],[2,2,0]], 'output': [[1,0],[0,2,2]]}]},
                             'arch': [{'op': 'get_largest'}, {'op': 'move', 'params': {'delta': (0,1)}}]},
            'delete_smallest': {'task': {'train': [{'input': [[1,0],[2,2,0]], 'output': [[0,0],[2,2,0]]}]},
                                'arch': [{'op': 'get_smallest'}, {'op': 'delete'}]},
        }

        # 3. Build HPN Level 1 (The "Playbook")
        playbook_vectors = []
        playbook_programs = []
        
        for name, data in QUIZ_TASKS.items():
            try:
                v = fingerprinter.fingerprint(data['task'])
                playbook_vectors.append(v)
                playbook_programs.append(data['arch'])
            except Exception as e:
                print(f"  ⚠️  Bootcamp HPN-L1 failed for {name}: {e}")
                continue
        
        final_playbook_vectors = None
        if playbook_vectors:
            final_playbook_vectors = np.stack(playbook_vectors)

        # 4. Build HPN Level 2 (The "Grammar")
        hpn_counts = defaultdict(lambda: defaultdict(int))
        for program_ast in CURRICULUM:
            prev_op = 'START'
            for instruction in program_ast:
                op_name = instruction['op']
                hpn_counts[prev_op][op_name] += 1
                prev_op = op_name
        
        hpn_grammar = defaultdict(dict)
        for prev_op, next_ops in hpn_counts.items():
            total_transitions = sum(next_ops.values())
            if total_transitions > 0:
                for next_op, count in next_ops.items():
                    hpn_grammar[prev_op][next_op] = count / total_transitions
        
        return final_playbook_vectors, playbook_programs, hpn_grammar

    def run_pre_computation(self, all_known_tasks, all_known_solutions):
        """
        *** LTM-v4 "GAME GENIE" (On-the-fly Training) ***
        *** HOTFIX 19: Passes `is_punt_task` flag. ***
        """
        print("\n--- 🧠 EXECUTING PRE-COMPUTATION: 'GAME GENIE' LTM-v4 TRAINING ---")
        if not all_known_tasks or not all_known_solutions:
            print("  ⚠️  Missing training/eval data. Skipping 'Game Genie' training.")
            return

        profiler.start("GameGenie_LTMv4_Training")
        self.metric_logger.write_header(self.LTM_LOG_COLUMNS)
        
        fingerprinter = self.solver_toolbox.fingerprinter
        total_tasks_to_train = len(all_known_tasks)
        tasks_cached = 0
        
        # --- HOTFIX 16: Dynamic Budget Calculation ---
        total_ltm_budget = self.config.total_time_budget * self.config.LTM_BUDGET_PERCENT
        
        if self.config.DIAGNOSTIC_RUN:
             diag_scaled_budget = 0.0
             if self.LTM_NUM_TASKS > 0:
                 diag_scaled_budget = (total_ltm_budget / self.LTM_NUM_TASKS) * total_tasks_to_train
             
             min_budget_seconds = self.config.DIAGNOSTIC_MIN_RUNTIME_MINUTES * 60
             total_ltm_budget = max(diag_scaled_budget, min_budget_seconds)
             
             print(f"  *** ⚠️  DIAGNOSTIC MODE: Training on {total_tasks_to_train} tasks... ***")
             print(f"  Scaled LTM Budget: {total_ltm_budget / 60:.2f} minutes ({self.config.DIAGNOSTIC_MIN_RUNTIME_MINUTES:.0f}-min floor enforced)")
        else:
             print(f"  Running 'Game Genie' LTM training on {total_tasks_to_train} tasks.")
             print(f"  Total LTM Budget: {total_ltm_budget / 3600:.2f} hours.")
        
        
        # --- HOTFIX 11: Phase 0 - Triage ALL tasks first ---
        print("  Phase 0: Triaging all tasks for curriculum...")
        profiler.start("GameGenie.TriageAll")
        sorted_task_list = []
        for task_id, task_data in all_known_tasks.items():
            if task_id not in all_known_solutions:
                continue
            difficulty = estimate_task_difficulty(task_data)
            tier = 'easy' if difficulty < 7.0 else ('medium' if difficulty < 18.0 else 'hard')
            sorted_task_list.append((task_id, task_data, difficulty, tier))
        
        sorted_task_list.sort(key=lambda x: x[2])
        profiler.end("GameGenie.TriageAll")
        
        
        # --- HOTFIX 16: "Shaving" Budget Calculation ---
        num_punt_tasks = 10
        num_standard_tasks = max(0, total_tasks_to_train - num_punt_tasks)
        
        punt_timeout = self.config.PUNT_TASK_BUDGET_SECONDS # 60.0s
        total_punt_cost = num_punt_tasks * punt_timeout # 10 * 60s = 600s
        
        remaining_budget = total_ltm_budget - total_punt_cost
        
        if num_standard_tasks > 0:
            standard_timeout = remaining_budget / num_standard_tasks
        else:
            standard_timeout = 0
        
        if standard_timeout < 1.0:
            if remaining_budget > 0:
                 print(f"  ⚠️  WARNING: Punt tasks consumed most of LTM budget. "
                       f"Standard tasks will have {standard_timeout:.2f}s timeout.")
            standard_timeout = max(1.0, standard_timeout) # 1s minimum
            
        
        tasks_to_punt = sorted_task_list[:num_punt_tasks]
        tasks_to_standard_solve = sorted_task_list[num_punt_tasks:]

        print(f"  Budgeting: Standard Timeout = {standard_timeout:.2f}s | Punt Timeout = {punt_timeout:.2f}s")
        
        
        # --- HOTFIX 19: Phase A - "Punt" Tasks (FORCED SLOW BRAIN) ---
        print(f"\n  --- Phase A: Solving {len(tasks_to_punt)} Easiest Tasks ({punt_timeout:.0f}s Budget) ---")
        tasks_processed = 0
        try:
            for (task_id, task_data, difficulty, tier) in tasks_to_punt:
                task_start_time = time.time()
                status = "Fail.Unknown"
                program_cached = False
                
                try:
                    ground_truth_outputs = all_known_solutions[task_id]
                    (validated_program_ast, status) = self._find_ground_truth_program(
                        task_id, task_data, ground_truth_outputs, 
                        timeout=punt_timeout, 
                        is_punt_task=True # <-- *** HOTFIX 19 FLAG ***
                    )
                    
                    if validated_program_ast is not None:
                        v_delta = fingerprinter.fingerprint(task_data)
                        self._ltm_vectors_list.append(v_delta)
                        self.ltm_programs.append(validated_program_ast)
                        program_cached = True
                        tasks_cached += 1
                
                except Exception as e:
                    status = f"Fail.Crash.{type(e).__name__}"
                finally:
                    time_taken = time.time() - task_start_time
                    self.metric_logger.log({
                        'columns_order': self.LTM_LOG_COLUMNS, 'phase': "GameGenie.Punt",
                        'task_id': task_id, 'task_tier': tier, 'status': status,
                        'time_taken_s': f"{time_taken:.3f}", 'program_cached': program_cached
                    })
                    tasks_processed += 1
                    print(f"  Punt Task {tasks_processed}/{len(tasks_to_punt)}: {task_id} "
                          f"({tier}) -> {status}. (Cached: {program_cached})")

        except Exception as e:
            print(f"  🔥🔥🔥 CRASH during LTM PUNT phase: {e}")


        # --- HOTFIX 19: Phase B - "Standard" Tasks (Fast -> Slow Fallback) ---
        print(f"\n  --- Phase B: Solving {len(tasks_to_standard_solve)} Remaining Tasks ({standard_timeout:.2f}s Budget) ---")
        try:
            for (task_id, task_data, difficulty, tier) in tasks_to_standard_solve:
                task_start_time = time.time()
                status = "Fail.Unknown"
                program_cached = False
                
                try:
                    ground_truth_outputs = all_known_solutions[task_id]
                    (validated_program_ast, status) = self._find_ground_truth_program(
                        task_id, task_data, ground_truth_outputs, 
                        timeout=standard_timeout, 
                        is_punt_task=False # <-- *** HOTFIX 19 FLAG ***
                    )
                    
                    if validated_program_ast is not None:
                        v_delta = fingerprinter.fingerprint(task_data)
                        self._ltm_vectors_list.append(v_delta)
                        self.ltm_programs.append(validated_program_ast)
                        program_cached = True
                        tasks_cached += 1
                
                except Exception as e:
                    status = f"Fail.Crash.{type(e).__name__}"
                finally:
                    time_taken = time.time() - task_start_time
                    self.metric_logger.log({
                        'columns_order': self.LTM_LOG_COLUMNS, 'phase': "GameGenie.Standard",
                        'task_id': task_id, 'task_tier': tier, 'status': status,
                        'time_taken_s': f"{time_taken:.3f}", 'program_cached': program_cached
                    })
                    tasks_processed += 1
                    
                    if (tasks_processed % 10 == 0) or (tasks_processed == total_tasks_to_train):
                        print(f"  LTM Training Progress... Task {tasks_processed + num_punt_tasks}/{total_tasks_to_train} processed. "
                              f"({tasks_cached} programs cached)")

        except Exception as e:
            print(f"  🔥🔥🔥 CRASH during LTM STANDARD phase: {e}")
            import traceback
            traceback.print_exc()

        # --- Finalize ---
        if self._ltm_vectors_list:
            self.ltm_vectors = np.stack(self._ltm_vectors_list)
            print(f"\n  ✅ 'Game Genie' LTM-v4 Training complete.")
            print(f"  Cached {self.ltm_vectors.shape[0]} programs in a "
                  f"({self.ltm_vectors.shape[0]} x {self.ltm_vectors.shape[1]}) vector database.")
        else:
            print(f"\n  ⚠️  'Game Genie' LTM-v4 Training FAILED. No programs were cached.")

        profiler.end("GameGenie_LTMv4_Training")

    def _find_ground_truth_program(self, task_id: str, task_data: Dict, 
                                   ground_truth_outputs: List[Dict], timeout: float,
                                   is_punt_task: bool = False # <-- *** HOTFIX 19 FLAG ***
                                   ) -> Tuple[Optional[List[Dict]], str]:
        """
        *** HOTFIX 19: Implements logic to *force* slow brain on punt tasks. ***
        """
        profiler.start(f"GameGenie.find_program.{task_id}")
        
        program_ast = None
        rule_name = "Fail.Unknown"

        # --- *** HOTFIX 19: New Logic *** ---
        # If it's *not* a punt task, we try the "Fast Brain" first.
        # If it *is* a punt task, we skip this and go straight to the "Slow Brain".
        if not is_punt_task:
            # --- 1. Call "Fast Brain" (The "Architect") ---
            fast_brain_timeout = min(timeout, 2.0) # Give 2s for a quick playbook check
            fast_solver = self.solver_toolbox.heuristic_solver
            program_ast, rule_name = fast_solver.solve(task_data, timeout=fast_brain_timeout)
            
            if program_ast is not None:
                # We found a match, now just validate it
                holdout_inputs = task_data.get('test', [])
                if self._validate_on_holdout(program_ast, holdout_inputs, ground_truth_outputs):
                    profiler.end(f"GameGenie.find_program.{task_id}")
                    return program_ast, rule_name # e.g., "Playbook.Success"
                else:
                    rule_name = "Playbook.Fail.Holdout"
                    program_ast = None # Clear it, force slow brain to run
        
        # --- 2. Call "Schooled Slow Brain" (The "Searcher") ---
        # This now runs if:
        # a) It's a "Punt" task (we skipped the fast brain).
        # b) It's a "Standard" task and the fast brain failed.
        if program_ast is None:
            slow_solver = self.solver_toolbox.synthesizer
            program_ast, rule_name = slow_solver.solve(task_data, timeout=timeout) 
        
        if program_ast is None:
            profiler.end(f"GameGenie.find_program.{task_id}")
            return None, rule_name # e.g., "Synthesizer.Fail.MaxDepth"
            
        # --- 3. Final Validation ---
        holdout_inputs = task_data.get('test', [])
        
        if self._validate_on_holdout(program_ast, holdout_inputs, ground_truth_outputs):
            profiler.end(f"GameGenie.find_program.{task_id}")
            return program_ast, rule_name # e.g., "Synthesizer.Success.d{N}"
        else:
            profiler.end(f"GameGenie.find_program.{task_id}")
            return None, "Synthesizer.Fail.Holdout"

    def _validate_on_holdout(self, program_ast: List[Dict], 
                               test_inputs: List[Dict], 
                               ground_truth_solutions: List[Dict]) -> bool:
        """ (Unchanged from HOTFIX 4) """
        interpreter = self.solver_toolbox.interpreter
        
        if len(test_inputs) != len(ground_truth_solutions):
            return False 
        if not test_inputs:
            return True 

        try:
            for i in range(len(test_inputs)):
                inp_grid = np.array(test_inputs[i]['input'])
                expected_output_grid = np.array(ground_truth_solutions[i]['output'])
                predicted_grid = interpreter.run(program_ast, inp_grid)
                
                if not np.array_equal(predicted_grid, expected_output_grid):
                    return False
            return True
        except Exception as e:
            return False

    def query_ltm_cache(self, novel_fingerprint: np.ndarray) -> List[Tuple[List[Dict], str, float]]:
        """ Performs a k-Nearest Neighbor (k-NN) search """
        profiler.start("query_ltm_cache")
        
        results = []
        if self.ltm_vectors is None or novel_fingerprint is None or len(self.ltm_programs) == 0:
            profiler.end("query_ltm_cache")
            return results
            
        try:
            distances = np.linalg.norm(self.ltm_vectors - novel_fingerprint, axis=1)
            k = self.config.LTM_CACHE_K
            nearest_indices = np.argsort(distances)[:k]
            
            for i in nearest_indices:
                program_ast = self.ltm_programs[i]
                dist = distances[i]
                name = f"LTM_Abduction_k{i}_dist{dist:.2f}"
                results.append( (program_ast, name, dist) )
                    
        except Exception as e:
            pass
            
        profiler.end("query_ltm_cache")
        return results

    def update_ltm_cache(self, fingerprint: np.ndarray, program_ast: List[Dict]):
        """ Performs "Online Learning". """
        profiler.start("update_ltm_cache")
        try:
            self._ltm_vectors_list.append(fingerprint)
            self.ltm_vectors = np.stack(self.ltm_vectors_list)
            self.ltm_programs.append(program_ast)
        except Exception as e:
            pass
        profiler.end("update_ltm_cache")

    def run_triage_phase(self, tasks_to_analyze: Dict[str, Dict]) -> Counter:
        """
        Executes "Phase 1 Triage" on the (unknown) TEST SET.
        (Unchanged from HOTFIX 11)
        """
        print("\n--- 🧠 EXECUTING PHASE 1: HEURISTIC TRIAGE (on test set) ---")
        if not tasks_to_analyze:
            print("  ❌ No tasks to analyze. Triage skipped.")
            return Counter()
            
        profiler.start("Phase1_Triage_Full")
        
        tasks_to_actually_triage = tasks_to_analyze
        if self.config.DIAGNOSTIC_RUN:
            print(f"  *** ⚠️  DIAGNOSTIC MODE: Triaging {self.config.DIAGNOSTIC_SAMPLE_SIZE} tasks... ***")
            task_keys_to_triage = list(tasks_to_analyze.keys())[:self.config.DIAGNOSTIC_SAMPLE_SIZE]
            tasks_to_actually_triage = {k: tasks_to_analyze[k] for k in task_keys_to_triage}
        
        for task_id, task_data in tasks_to_actually_triage.items():
            profile = self.solver_toolbox.analyzer.analyze(task_data, task_id)
            self.task_profiles[task_id] = profile
            
        profiler.end("Phase1_Triage_Full")
        
        stats = Counter([p.difficulty_tier for p in self.task_profiles.values()])
        print(f"  ✅ Triage complete. Task profile generated for all {len(self.task_profiles)} tasks.")
        print(f"  Triage Stats: {stats['easy']} Easy | {stats['medium']} Medium | {stats['hard']} Hard")
        return stats

    def allocate_time_budgets(self, total_solve_budget: float, tier_counts: Counter):
        """
        *** HOTFIX 16: Implements "60-second punt" for INFERENCE. ***
        """
        print("\n--- ⏳ Allocating time budgets (ΔH Modulation) ---")
        
        total_abstraction_budget = total_solve_budget * self.config.abstraction_pass_time_ratio
        total_reasoning_budget = total_solve_budget * self.config.reasoning_pass_time_ratio
        
        print(f"  Abstraction Pass (ΔH-) Total Budget: {total_abstraction_budget/3600:.2f} hours")
        print(f"  Reasoning Pass (ΔH+) Total Budget: {total_reasoning_budget/3600:.2f} hours")

        total_tasks = len(self.task_profiles)
        if total_tasks == 0:
            print("  ❌ No tasks found. Cannot allocate time.")
            return

        # --- 1. Identify the 9 "Punt" Tasks (3 easy, 3 med, 3 hard) ---
        sorted_profiles = sorted(self.task_profiles.values(), key=lambda p: p.difficulty_score)
        
        punt_tasks_easy = [p.task_id for p in sorted_profiles if p.difficulty_tier == 'easy'][:3]
        punt_tasks_medium = [p.task_id for p in sorted_profiles if p.difficulty_tier == 'medium'][:3]
        punt_tasks_hard = [p.task_id for p in sorted_profiles if p.difficulty_tier == 'hard'][:3]
        
        punt_task_set = set(punt_tasks_easy + punt_tasks_medium + punt_tasks_hard)
        print(f"  Identified {len(punt_task_set)} tasks for 2x budget 'punt' (60s).")

        # --- 2. Calculate "Shaved" Budget (MDMP) ---
        weights = {'easy': 1, 'medium': 3, 'hard': 6}
        punt_timeout = self.config.PUNT_TASK_BUDGET_SECONDS # 60.0s
        
        punt_cost_reasoning = len(punt_task_set) * punt_timeout
        punt_cost_abstraction = len(punt_task_set) * 10.0 # 10s for LTM/HPN check
        
        remaining_budget_reasoning = total_reasoning_budget - punt_cost_reasoning
        remaining_budget_abstraction = total_abstraction_budget - punt_cost_abstraction
        
        remaining_weighted_units = 0
        for task_id, profile in self.task_profiles.items():
            if task_id not in punt_task_set:
                remaining_weighted_units += weights.get(profile.difficulty_tier, 1)

        if remaining_weighted_units <= 0: remaining_weighted_units = 1
        
        unit_time_abstraction = remaining_budget_abstraction / remaining_weighted_units
        unit_time_reasoning = remaining_budget_reasoning / remaining_weighted_units
        
        if unit_time_reasoning < 1.0 or unit_time_abstraction < 1.0:
            print(f"  ⚠️  WARNING: 60s punt tasks consumed entire budget (Remaining: {remaining_budget_reasoning:.0f}s). "
                  f"All other tasks will get 1s.")
            unit_time_abstraction = max(1.0, unit_time_abstraction)
            unit_time_reasoning = max(1.0, unit_time_reasoning)

        # --- 4. Assign Budgets (Punt or Shaved) ---
        for task_id, profile in self.task_profiles.items():
            if task_id in punt_task_set:
                self.time_allocations['abstraction_per_task'][task_id] = 10.0
                self.time_allocations['reasoning_per_task'][task_id] = punt_timeout
            else:
                base_weight = weights.get(profile.difficulty_tier, 1)
                self.time_allocations['abstraction_per_task'][task_id] = unit_time_abstraction * base_weight
                self.time_allocations['reasoning_per_task'][task_id] = unit_time_reasoning * base_weight
        
        print("  ✅ Time budgets allocated per task (60s Punt Strategy).")
        std_easy_task_id = next((p.task_id for p in sorted_profiles if p.difficulty_tier == 'easy' and p.task_id not in punt_task_set), None)
        if std_easy_task_id:
             print(f"  Time (Std. Easy, Rea): {self.time_allocations['reasoning_per_task'].get(std_easy_task_id, 0.0):.1f}s")
        if punt_tasks_easy:
            print(f"  Time (Punt Easy, Rea): {self.time_allocations['reasoning_per_task'].get(punt_tasks_easy[0], 0.0):.1f}s (PUNT)")
        
        std_hard_task_id = next((p.task_id for p in reversed(sorted_profiles) if p.difficulty_tier == 'hard' and p.task_id not in punt_task_set), None)
        if std_hard_task_id:
             print(f"  Time (Std. Hard, Rea): {self.time_allocations['reasoning_per_task'].get(std_hard_task_id, 0.0):.1f}s")
        if punt_tasks_hard:
             print(f"  Time (Punt Hard, Rea): {self.time_allocations['reasoning_per_task'].get(punt_tasks_hard[0], 0.0):.1f}s (PUNT)")


# --- 2. Main Initialization & "Game Genie" Execution ---

notebook_start_time = time.time()
setup_memory_limits() # From Cell 2

# Instantiate the Master Controller, which builds the whole AGI (Cell 7)
profiler.start("Total_Initialization")
controller = RSC_Controller(CONFIG)
profiler.end("Total_Initialization")


# --- THIS IS THE LTM-v4 "TRAINING" STEP ---
if ('training_tasks' in locals() and 'evaluation_tasks' in locals() and
    'training_solutions' in locals() and 'evaluation_solutions' in locals()):
    
    all_known_tasks = {**training_tasks, **evaluation_tasks}
    all_known_solutions = {**training_solutions, **evaluation_solutions}
    
    tasks_to_train_on = all_known_tasks
    solutions_to_train_on = all_known_solutions
    
    if CONFIG.DIAGNOSTIC_RUN:
        print("--- ⚠️  DIAGNOSTIC MODE: Sampling training data... ---")
        sample_size = CONFIG.DIAGNOSTIC_SAMPLE_SIZE
        sample_keys = list(all_known_tasks.keys())[:sample_size]
        
        tasks_to_train_on = {k: all_known_tasks[k] for k in sample_keys}
        solutions_to_train_on = {k: all_known_solutions[k] for k in sample_keys if k in all_known_solutions}
        print(f"--- ⚠️  Running 'Game Genie' on {len(tasks_to_train_on)} tasks. ---")

    controller.run_pre_computation(
        tasks_to_train_on,
        solutions_to_train_on
    )
else:
    print("\n⚠️  Missing training/evaluation data. Skipping 'Game Genie' LTM-v4 training.")


# --- Run Phase 1 Triage on the TEST SET ---
if test_tasks:
    tier_stats = controller.run_triage_phase(test_tasks)
    
    init_time_taken = time.time() - notebook_start_time
    total_budget = CONFIG.total_time_budget
    submission_buffer = CONFIG.submission_buffer
    
    solve_budget = total_budget - init_time_taken - submission_buffer
    
    if CONFIG.DIAGNOSTIC_RUN:
        min_runtime_seconds = CONFIG.DIAGNOSTIC_MIN_RUNTIME_MINUTES * 60
        # Enforce 30-min floor
        if (init_time_taken + solve_budget + submission_buffer) < min_runtime_seconds:
            solve_budget = max(0.0, min_runtime_seconds - init_time_taken - submission_buffer)
            print(f"  DIAGNOSTIC 30-min floor: New Solve Budget is {solve_budget/60:.2f} minutes.")
    
    print(f"\nInitialization & LTM-v4 Training took {init_time_taken / 60:.2f} minutes.")
    print(f"Total Solve Budget Remaining: {solve_budget/3600:.2f} hours.")

    controller.allocate_time_budgets(solve_budget, tier_stats)
else:
    print("\n❌ No test tasks loaded. Skipping Triage and Time Allocation.")
    solve_budget = 0 

print("="*70)
#Cell 9


🌊⚛️ LucidOrca Solver: Cell 9 (LTM-v4) RSC Controller & LTM Trainer [HOTFIX 19]
🧠 Memory limit set: 12.00 GB
♻️  Garbage collection: ENABLED (aggressive mode)

Initializing RSC_Controller (LTM-v4)...
  ✅ MetricLogger initialized. Writing to /kaggle/working/lucid_metrics.csv
  Bootstrapping Cognitive Architecture (Phase 1)...

🌊⚛️ Initializing LucidOrcaUltimateSolver (LTM-v4 Master Toolbox)...
  PerceptionEngine (LTM-v4 'Vision') initialized.
  ✅ 1. PerceptionEngine (Vision)... LOADED
  VisionModelEncoder (LTM-v4) initialized. Fingerprint dim: 28
  TaskFingerprinter (Task-to-Delta-Vector Engine) initialized.
  TaskAnalyzer (Phase 1 Triage Engine) initialized.
  ✅ 2. TaskAnalyzer (Triage & Fingerprinting)... LOADED
  MetaPrimitives (LTM-v4 'ALU') initialized. [HOTFIX 15]
  SymbolicProgramInterpreter (LTM-v4 'CPU') initialized. [HOTFIX 19]
  ✅ 3. SymbolicProgramInterpreter ('CPU')... LOADED
  HeuristicPlaybookSolver (LTM-v4 'Fast Brain') initialized. [HPN *NOT* LOADED]
  ✅ 4. HeuristicPlay

In [10]:
#Cell 10
################################################################################
#
# 🌊⚛️ LUCIDORCA ULTIMATE SOLVER - (LTM-v4 REBUILD)
#
# Cell 10: Main Inference, Validation, and Save
#
# *** LTM-v4 HOTFIX 16 (The "Two-Brain" Solution): ***
#
# 1. The main inference loop is re-architected to use our "Two-Brain"
#    model, separating "Abstraction" from "Reasoning".
# 2. Phase 2 (Abstraction): We now call the new "Fast Brain"
#    (`heuristic_solver`). This runs the HPN Playbook k-NN search.
# 3. Phase 3 (Reasoning): *Only if Phase 2 fails*, we call the
#    "Slow Brain" (`synthesizer`) to run its brute-force search.
# 4. This is a true "Heuristics-first" cognitive architecture.
#
################################################################################

print("="*70)
print("🌊⚛️ LucidOrca Solver: Cell 10 (LTM-v4) - Main Inference Loop [HOTFIX 16]")

# --- 1. Setup Execution ---

submission = {}
DEFAULT_PROGRAM_AST = []

def _generate_variation_grid(grid: np.ndarray) -> np.ndarray:
    """
    Generates a *different* grid to be used for `attempt_2`
    if both solver passes return the *exact same* grid.
    """
    if grid is None or grid.size == 0:
        return np.array([[0]])

    try:
        variation_1 = np.rot90(grid, 1)
        if not np.array_equal(variation_1, grid):
            return variation_1
        
        variation_2 = np.fliplr(grid)
        if not np.array_equal(variation_2, grid):
            return variation_2
            
        return np.rot90(grid, 2)
        
    except Exception:
        return np.array([[0]])


def _validate_program_on_train(program_ast: List[Dict], 
                             task_data: Dict, 
                             interpreter: SymbolicProgramInterpreter) -> bool:
    """
    Validates a "best guess" program (from LTM) against the
    novel task's *own* training examples.
    """
    if program_ast is None:
        return False
        
    examples = task_data.get('train', [])
    if not examples:
        return False

    try:
        for ex in examples:
            inp_grid = np.array(ex['input'])
            expected_output_grid = np.array(ex['output'])
            predicted_grid = interpreter.run(program_ast, inp_grid)
            
            if not np.array_equal(predicted_grid, expected_output_grid):
                return False
        return True
    except Exception:
        return False


# Check that the controller and tasks from previous cells are loaded
if 'controller' not in locals() or 'test_tasks' not in locals() or not test_tasks:
    print("❌ CRITICAL ERROR: `controller` or `test_tasks` not found.")
    print("   Please re-run Cell 8 and Cell 9 before this one.")
    
else:
    # --- 2. Get All AGI Components from Controller ---
    
    solver_toolbox = controller.solver_toolbox
    interpreter = solver_toolbox.interpreter
    
    # --- *** NEW (HOTFIX 16): Get *both* brains *** ---
    fast_brain_solver = solver_toolbox.heuristic_solver # "The Architect"
    slow_brain_solver = solver_toolbox.synthesizer      # "The Searcher"
    
    task_profiles = controller.task_profiles
    time_allocations = controller.time_allocations
    
    total_solve_budget = solve_budget 
    main_loop_start_time = time.time()

    sorted_task_ids = list(task_profiles.keys())
    total_tasks_count = len(sorted_task_ids)
    
    print(f"  Starting main LTM-v4 inference loop for {total_tasks_count} tasks...")
    if CONFIG.DIAGNOSTIC_RUN:
        print(f"  *** ⚠️  DIAGNOSTIC MODE: Inference run is limited to {total_tasks_count} tasks. ***")
    print(f"  Total Solve Budget: {total_solve_budget/3600:.2f} hours")
    print("="*70)

    # --- HOTFIX 9: Write new header for Inference phase ---
    controller.metric_logger.write_header(controller.INFERENCE_LOG_COLUMNS)

    # --- 3. Main Execution Loop (LTM-v4 Inference) ---
    
    for i, task_id in enumerate(sorted_task_ids):
        task_start_time = time.time()
        
        task_data = test_tasks[task_id]
        profile = task_profiles[task_id]
        
        elapsed_total = time.time() - main_loop_start_time
        if elapsed_total > total_solve_budget:
            print(f"\n⏱️  MASTER TIME BUDGET EXCEEDED. Stopping solve loop.")
            print(f"  Completed {i}/{total_tasks_count} tasks.")
            break
            
        print(f"\n--- [ {i+1}/{total_tasks_count} ] Solving Task: {task_id} (Tier: {profile.difficulty_tier}, Basin: {profile.basin}) ---")
        
        program_LTM = None # "Abstraction" (LTM k-NN)
        program_Heuristic = None # "Architect" (HPN Playbook)
        program_DeepSearch = None # "Searcher" (Brute-Force)

        ltm_status = "Miss"
        heuristic_status = "Fail.NotRun"
        reasoning_status = "Fail.NotRun"
        
        # --- C. Execute Phase 2: Abstraction (LTM k-NN) ---
        # This is our *real* LTM, which we are *finally* populating.
        profiler.start(f"Phase2_LTM_Query.{task_id}")
        
        novel_fingerprint = profile.delta_fingerprint
        
        if novel_fingerprint is None:
            print("  ⚠️  Task fingerprinting failed. Skipping LTM query.")
            ltm_status = "Fail.NoFingerprint"
        else:
            print("  Querying LTM-v4 (k-NN)...\n  (This is the *real* LTM, not the HPN)")
            candidates = controller.query_ltm_cache(novel_fingerprint)
            
            if candidates:
                print(f"  LTM-v4 returned {len(candidates)} candidates. Starting Sanity Check...")
                for (program_ast, rule_name, dist) in candidates:
                    if _validate_program_on_train(program_ast, task_data, interpreter):
                        print(f"  ✅ Abduction PASSED Sanity Check! (Found {rule_name})")
                        program_LTM = program_ast
                        ltm_status = "Hit"
                        controller.update_ltm_cache(novel_fingerprint, program_LTM)
                        break 
                if program_LTM is None:
                    print("  ⚠️  LTM Abduction FAILED Sanity Check on all candidates.")
                    ltm_status = "Fail.SanityCheck"
            else:
                print("  LTM-v4 Cache Miss. No similar task found.")
                ltm_status = "Miss"
        profiler.end(f"Phase2_LTM_Query.{task_id}")

        # --- D. Execute Phase 3a: Heuristics ("Fast Brain") ---
        # If the *real* LTM missed, we try our "Fast Brain" architect.
        if program_LTM is None:
            profiler.start(f"Phase3a_HeuristicSolver.{task_id}")
            print("  Executing Phase 3a (ΔH-) Fast-Brain Architect...")
            
            abstraction_budget = time_allocations['abstraction_per_task'][task_id]
            (program_ast, rule_name) = fast_brain_solver.solve(task_data, timeout=abstraction_budget)
            
            if program_ast is not None:
                program_Heuristic = program_ast
                heuristic_status = rule_name # e.g., "Playbook.Success"
                print(f"  ✅ Phase 3a (ΔH-) Fast-Brain SUCCESS: [{heuristic_status}]")
            else:
                heuristic_status = rule_name # e.g., "Playbook.Fail.NoMatch"
                print(f"  ⚠️  Phase 3a (ΔH-) Fast-Brain FAILED: [{heuristic_status}]")
            profiler.end(f"Phase3a_HeuristicSolver.{task_id}")

        # --- E. Execute Phase 3b: Reasoning ("Slow Brain") ---
        # If *both* LTM and Fast Brain failed, we fall back to brute-force.
        if program_LTM is None and program_Heuristic is None:
            profiler.start(f"Phase3b_DeepSearch.{task_id}")
            print("  Executing Phase 3b (ΔH+) Slow-Brain Search...")
            
            reasoning_budget = time_allocations['reasoning_per_task'][task_id]
            (program_ast, rule_name) = slow_brain_solver.solve(task_data, timeout=reasoning_budget)
            
            if program_ast is not None:
                program_DeepSearch = program_ast
                reasoning_status = rule_name # e.g., "Synthesizer.Success.d3"
                print(f"  ✅ Phase 3b (ΔH+) Slow-Brain SUCCESS: [{reasoning_status}]")
            else:
                reasoning_status = rule_name # e.g., "Synthesizer.Fail.MaxDepth"
                print(f"  ⚠️  Phase 3b (ΔH+) Slow-Brain FAILED: [{reasoning_status}]")
            profiler.end(f"Phase3b_DeepSearch.{task_id}")


        # --- F. RSC Arbiter: Select Final Programs ---
        final_program_1 = program_LTM or program_Heuristic or program_DeepSearch or DEFAULT_PROGRAM_AST
        final_program_2 = program_DeepSearch or program_Heuristic or program_LTM or DEFAULT_PROGRAM_AST
        
        final_program_source = "Fallback"
        if program_LTM: final_program_source = "LTM"
        elif program_Heuristic: final_program_source = "Heuristic"
        elif program_DeepSearch: final_program_source = "DeepSearch"
        
        
        # --- G. CRITICAL: Apply Programs to ALL Test Cases ---
        task_solutions = [] 
        num_expected_outputs = len(task_data.get('test', []))
        
        if num_expected_outputs == 0:
            print(f"  ❌ Task {task_id} has no test cases. Skipping.")
            continue
            
        print(f"  Applying programs to {num_expected_outputs} test case(s)...")
        
        for test_case_index in range(num_expected_outputs):
            try:
                test_input_grid = np.array(task_data['test'][test_case_index]['input'])
            except Exception:
                test_input_grid = np.array([[0]]) 
            
            try:
                grid_1 = interpreter.run(final_program_1, test_input_grid)
            except Exception:
                grid_1 = test_input_grid
                
            try:
                grid_2 = interpreter.run(final_program_2, test_input_grid)
            except Exception:
                grid_2 = test_input_grid
                
            if np.array_equal(grid_1, grid_2):
                grid_2 = _generate_variation_grid(grid_1)
                
            task_solutions.append({
                "attempt_1": grid_1.tolist(),
                "attempt_2": grid_2.tolist()
            })

        # --- H. Store Final Solutions for this Task ---
        submission[task_id] = task_solutions
        task_time = time.time() - task_start_time
        print(f"  ➡️  Task {task_id} finished in {task_time:.2f}s. Stored {len(task_solutions)} solutions.")

        # --- HOTFIX 9: Log Inference Metrics ---
        log_data = {
            'columns_order': controller.INFERENCE_LOG_COLUMNS,
            'phase': "Inference",
            'task_id': task_id,
            'task_tier': profile.difficulty_tier,
            'basin': profile.basin,
            'ltm_status': ltm_status,
            'reasoning_status': f"{heuristic_status} | {reasoning_status}",
            'final_program': final_program_source,
            'time_taken_s': f"{task_time:.3f}"
        }
        controller.metric_logger.log(log_data)


    # --- 4. Final Summary of Loop ---
    
    total_solve_time = time.time() - main_loop_start_time
    print("\n" + "="*70)
    print("✅ MAIN INFERENCE LOOP COMPLETE")
    print(f"  Total tasks processed: {len(submission)} / {total_tasks_count}")
    print(f"  Total Solve Time: {total_solve_time / 60:.2f} minutes")
    profiler.print_summary()


# --- 5. Final Validation & Sanitization ---

print("\n" + "="*70)
print("🌊⚛️ Final Validation & Sanitization Pass")
print("="*70)

def generate_compliant_fallback(task_id: str, task_data: Dict) -> List[Dict]:
    """
    Generates a compliant fallback solution (copies input)
    for a given task_id.
    """
    try:
        fallback_grid = np.array(task_data['test'][0]['input'])
    except Exception:
        fallback_grid = np.array([[0]]) 

    num_expected_outputs = len(task_data.get('test', []))
    if num_expected_outputs == 0:
        return [] 

    all_test_solutions = []
    for test_case_index in range(num_expected_outputs):
        try:
            input_grid = np.array(task_data['test'][test_case_index]['input'])
        except Exception:
            input_grid = fallback_grid 
            
        attempt_1_grid = input_grid
        attempt_2_grid = _generate_variation_grid(attempt_1_grid)
        
        all_test_solutions.append({
            "attempt_1": attempt_1_grid.tolist(),
            "attempt_2": attempt_2_grid.tolist()
        })
    
    return all_test_solutions


validation_passed = True
tasks_overwritten = 0
tasks_missing = 0

if not test_tasks:
    print("❌ No test tasks loaded. Cannot validate or save.")
    validation_passed = False
else:
    print(f"  Validating submission against all {len(test_tasks)} required tasks...")
    
    for task_id, task_data in test_tasks.items():
        
        num_expected_outputs = len(task_data.get('test', []))
        if num_expected_outputs == 0:
            continue 

        is_valid = True 

        if task_id not in submission:
            tasks_missing += 1
            is_valid = False
            
        else:
            task_outputs_list = submission[task_id]
            
            if not isinstance(task_outputs_list, list): is_valid = False
            elif len(task_outputs_list) != num_expected_outputs: is_valid = False
            else:
                for item in task_outputs_list:
                    if not isinstance(item, dict): is_valid = False; break
                    if "attempt_1" not in item or "attempt_2" not in item: is_valid = False; break
                    if not isinstance(item['attempt_1'], list) or not isinstance(item['attempt_2'], list): is_valid = False; break

        if not is_valid:
            if task_id in submission:
                print(f"  ❌ Task {task_id} has FORMAT ERROR. Overwriting with compliant fallback.")
                tasks_overwritten += 1
            
            submission[task_id] = generate_compliant_fallback(task_id, task_data)
            validation_passed = False

if tasks_missing > 0:
    print(f"\n! VALIDATION WARNING: {tasks_missing} tasks were missing and replaced with fallbacks.")
if tasks_overwritten > 0:
    print(f"\n! VALIDATION ERROR: {tasks_overwritten} tasks had format errors and were overwritten.")
if validation_passed and tasks_missing == 0 and tasks_overwritten == 0:
    print("\n✅✅✅ SUBMISSION FORMAT IS 100% VALID! ✅✅✅")
else:
    print("\n⚠️  Submission contains fallbacks or errors, but is now 100% format-compliant.")


# --- 6. Save the Sanitized Submission ---

OUTPUT_PATH = Path("/kaggle/working/submission.json")
print(f"\n💾 Saving sanitized submission to: {OUTPUT_PATH}")

try:
    with open(OUTPUT_PATH, 'w') as f:
        json.dump(submission, f, separators=(',', ':'))

    file_size = OUTPUT_PATH.stat().st_size
    print(f"   ✅ Saved {len(submission)} tasks ({file_size/1024:.1f} KB)")

except Exception as e:
    print(f"❌ CRITICAL ERROR: FAILED TO SAVE SUBMISSION: {e}")

# --- 7. Final Notebook Summary ---

total_notebook_time = time.time() - notebook_start_time
print("\n" + "="*70)
print("🏆🏁 NOTEBOOK EXECUTION COMPLETE 🏁🏆")
print("="*70)
print(f"  Total Tasks in Submission: {len(submission)} / {len(test_tasks)}")
print(f"  Total Notebook Runtime: {total_notebook_time / 60:.2f} minutes")
print(f"  (Total Budget: {CONFIG.total_time_budget / 60:.2f} minutes)")
print(f"\n  Final File: {OUTPUT_PATH}")
if 'file_size' in locals():
    print(f"  Final Size: {file_size/1024:.1f} KB")

# --- HOTFIX 9: Close the logger ---
if 'controller' in locals() and hasattr(controller, 'metric_logger'):
    controller.metric_logger.close()

print("\n  Ready for Kaggle submission.")
print("="*70)
#Cell 10


🌊⚛️ LucidOrca Solver: Cell 10 (LTM-v4) - Main Inference Loop [HOTFIX 16]
  Starting main LTM-v4 inference loop for 100 tasks...
  *** ⚠️  DIAGNOSTIC MODE: Inference run is limited to 100 tasks. ***
  Total Solve Budget: 7.71 hours

--- [ 1/100 ] Solving Task: 00576224 (Tier: hard, Basin: scaling) ---
  Querying LTM-v4 (k-NN)...
  (This is the *real* LTM, not the HPN)
  LTM-v4 Cache Miss. No similar task found.
  Executing Phase 3a (ΔH-) Fast-Brain Architect...
  ⚠️  Phase 3a (ΔH-) Fast-Brain FAILED: [Playbook.Fail.NoMatch]
  Executing Phase 3b (ΔH+) Slow-Brain Search...
  ⚠️  Phase 3b (ΔH+) Slow-Brain FAILED: [Synthesizer.Fail.MaxDepth]
  Applying programs to 1 test case(s)...
  ➡️  Task 00576224 finished in 0.40s. Stored 1 solutions.

--- [ 2/100 ] Solving Task: 007bbfb7 (Tier: hard, Basin: scaling) ---
  Querying LTM-v4 (k-NN)...
  (This is the *real* LTM, not the HPN)
  LTM-v4 Cache Miss. No similar task found.
  Executing Phase 3a (ΔH-) Fast-Brain Architect...
  ⚠️  Phase 3a (ΔH-) 